# Mapping Medi-Cal Deserts in California: Medi-Cal Intensity, Primary Care Capacity, Shortage Designations, and Preventable Hospitalizations (PQI)

## Project Overview

This notebook builds a county-year panel for California that links:
1. Medi-Cal enrollment intensity (need/exposure)
2. Primary care access proxies: physician supply (patient-care hours dataset) + primary care shortage designation
3. Outcomes: preventable hospitalizations (AHRQ PQI by county-year)
4. Controls: ACS demographics/economic/education covariates

**Main Questions:**
1. Where are Medi-Cal "deserts" in California (high Medi-Cal share + low primary care capacity and/or shortage designation)?
2. Are deserts associated with worse outcomes (higher preventable hospitalization rates)?
3. Do changes in Medi-Cal intensity correlate with changes in outcomes and capacity over time?

**Scope:** California counties only (58 counties)

## 0) Setup & Reproducibility

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
from pathlib import Path
import re

# Statistical modeling
try:
    from linearmodels import PanelOLS
    HAS_PANELOLS = True
except ImportError:
    HAS_PANELOLS = False
    print("WARNING: linearmodels not available. Will use statsmodels with dummies.")
    from statsmodels.regression.linear_model import OLS
    import statsmodels.api as sm

# Optional: geopandas for maps (only if shapefile exists)
try:
    import geopandas as gpd
    HAS_GEOPANDAS = True
except ImportError:
    HAS_GEOPANDAS = False
    print("WARNING: geopandas not available. Maps will be skipped.")

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 200)

print("Setup complete.")

In [ ]:
# Create output directories
output_dirs = ['outputs', 'outputs/data', 'outputs/figures', 'outputs/tables']
for dir_path in output_dirs:
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created/verified: {dir_path}/")

In [ ]:
# Helper Functions

def standardize_county_name(name):
    """Standardize county name for merging."""
    if pd.isna(name):
        return None
    name = str(name).upper()
    # Remove "County, California" and similar suffixes
    name = re.sub(r',?\s*CALIFORNIA\s*$', '', name)
    name = re.sub(r'\s+COUNTY\s*$', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    # Remove punctuation
    name = re.sub(r'[^\w\s]', '', name)
    return name

def make_fips5(state_fips, county_fips3):
    """Create 5-digit FIPS code."""
    state_str = str(int(state_fips)).zfill(2)
    county_str = str(int(county_fips3)).zfill(3)
    return state_str + county_str

def safe_read_csv(filepath, **kwargs):
    """Safely read CSV with error handling."""
    try:
        return pd.read_csv(filepath, **kwargs)
    except Exception as e:
        print(f"ERROR reading {filepath}: {e}")
        raise

def safe_read_excel(filepath, **kwargs):
    """Safely read Excel with error handling."""
    try:
        return pd.read_excel(filepath, **kwargs)
    except Exception as e:
        print(f"ERROR reading {filepath}: {e}")
        raise

def assert_expected_columns(df, expected_cols, filepath):
    """Check that expected columns exist."""
    missing = set(expected_cols) - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns in {filepath}: {missing}")
    print(f"✓ All expected columns present in {filepath}")

def summarize_missingness(df, name="Dataset"):
    """Print missingness summary."""
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    summary = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Pct': missing_pct
    })
    summary = summary[summary['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
    if len(summary) > 0:
        print(f"\nMissingness in {name}:")
        print(summary)
    else:
        print(f"\n✓ No missing values in {name}")
    return summary

def merge_report(left, right, on, how='inner', left_name='Left', right_name='Right'):
    """Report merge results."""
    merged = pd.merge(left, right, on=on, how=how, indicator=True)
    print(f"\nMerge: {left_name} + {right_name} on {on} ({how})")
    print(f"  Left rows: {len(left)}")
    print(f"  Right rows: {len(right)}")
    print(f"  Merged rows: {len(merged)}")
    if '_merge' in merged.columns:
        merge_counts = merged['_merge'].value_counts()
        print(f"  Merge status:")
        for status, count in merge_counts.items():
            print(f"    {status}: {count}")
    return merged

print("Helper functions defined.")

## 1) Load Raw Data

In [ ]:
# Required files - Updated to match actual data structure
# Note: county name.xlsx is not a crosswalk - we'll build crosswalk from ACS data
REQUIRED_FILES = [
    'medi-cal-enrollment-dashboard-data.csv',
    'E4 estiamtes.xlsx',
    'physicians-actively-working-by-specialty-and-patient-care-hours.xlsx',
    'Primary CAre Shortage .csv',
    'PQI.csv',
    'demoACS.csv',
    'educACS.csv',
    'EconACS.csv'
]

# Check for required files
missing_files = []
for f in REQUIRED_FILES:
    if not os.path.exists(f):
        missing_files.append(f)
    else:
        print(f"✓ Found: {f}")

if missing_files:
    raise FileNotFoundError(f"Missing required files: {missing_files}")

print(f"\n✓ All {len(REQUIRED_FILES)} required files found.")

In [ ]:
# Load required files
print("Loading required files...\n")

# Medi-Cal enrollment
df_medical_dash = safe_read_csv('medi-cal-enrollment-dashboard-data.csv', low_memory=False)
print(f"medi-cal-enrollment-dashboard-data.csv: {df_medical_dash.shape}")
print(f"Columns: {list(df_medical_dash.columns)}\n")

# Population estimates - Load from correct sheet with proper headers
df_e4_raw = pd.read_excel('E4 estiamtes.xlsx', sheet_name='Table 1 County State', header=None)
print(f"E4 estiamtes.xlsx (raw): {df_e4_raw.shape}")
print(df_e4_raw.head(10))
print()

# Physician supply
df_physician = safe_read_excel('physicians-actively-working-by-specialty-and-patient-care-hours.xlsx')
print(f"physicians-actively-working-by-specialty-and-patient-care-hours.xlsx: {df_physician.shape}")
print(df_physician.head())
print(f"Columns: {list(df_physician.columns)}\n")

# Shortage designation
df_shortage = safe_read_csv('Primary CAre Shortage .csv', low_memory=False)
print(f"Primary CAre Shortage .csv: {df_shortage.shape}")
print(f"Columns: {list(df_shortage.columns)}\n")

# PQI outcomes
df_pqi = safe_read_csv('PQI.csv', low_memory=False)
print(f"PQI.csv: {df_pqi.shape}")
print(f"Columns: {list(df_pqi.columns)}\n")

# ACS controls
df_demo = safe_read_csv('demoACS.csv')
print(f"demoACS.csv: {df_demo.shape}")
print(f"Columns: {list(df_demo.columns)}\n")

df_educ = safe_read_csv('educACS.csv')
print(f"educACS.csv: {df_educ.shape}")
print(f"Columns: {list(df_educ.columns)}\n")

df_econ = safe_read_csv('EconACS.csv')
print(f"EconACS.csv: {df_econ.shape}")
print(f"Columns: {list(df_econ.columns)}\n")

In [ ]:
# Optional files
optional_files = {
    'Medi-CAL.csv': None,
    'pqi-physicians-specialities-list.xlsx': None,
    'pqi-physicians-data-dictionary.csv': None,
    'pqi-physicians-data-set.csv': None,
    'physicians-actively-working-by-specialty-and-administration-activity-hours.xlsx': None,
    'physicians-actively-working-by-specialty-and-other-activity-hours.xlsx': None,
    'physicians-actively-working-by-specialty-and-research-activity-hours.xlsx': None,
    'physicians-actively-working-by-specialty-and-training-activity-hours.xlsx': None
}

for fname in optional_files.keys():
    if os.path.exists(fname):
        try:
            if fname.endswith('.csv'):
                optional_files[fname] = safe_read_csv(fname, low_memory=False)
            else:
                optional_files[fname] = safe_read_excel(fname)
            print(f"✓ Loaded optional: {fname} ({optional_files[fname].shape})")
        except Exception as e:
            print(f"⚠ Could not load optional {fname}: {e}")
    else:
        print(f"⊘ Optional file not found: {fname} (will skip)")

# Store specialty list if available
df_specialty_list = optional_files.get('pqi-physicians-specialities-list.xlsx')

## 2) Build Clean County Crosswalk

In [ ]:
# Build county crosswalk from ACS data (which has county names and FIPS codes)
# The ACS files have: NAME, state, county columns where state=6 (CA) and county is 3-digit FIPS

print("Building county crosswalk from ACS data...")
print(f"ACS demo data: {df_demo.shape}")
print(df_demo.head())

# California FIPS codes - official list (58 counties)
# Standard California county FIPS codes
CA_COUNTY_FIPS = {
    'ALAMEDA': '001', 'ALPINE': '003', 'AMADOR': '005', 'BUTTE': '007', 'CALAVERAS': '009',
    'COLUSA': '011', 'CONTRA COSTA': '013', 'DEL NORTE': '015', 'EL DORADO': '017', 'FRESNO': '019',
    'GLENN': '021', 'HUMBOLDT': '023', 'IMPERIAL': '025', 'INYO': '027', 'KERN': '029',
    'KINGS': '031', 'LAKE': '033', 'LASSEN': '035', 'LOS ANGELES': '037', 'MADERA': '039',
    'MARIN': '041', 'MARIPOSA': '043', 'MENDOCINO': '045', 'MERCED': '047', 'MODOC': '049',
    'MONO': '051', 'MONTEREY': '053', 'NAPA': '055', 'NEVADA': '057', 'ORANGE': '059',
    'PLACER': '061', 'PLUMAS': '063', 'RIVERSIDE': '065', 'SACRAMENTO': '067', 'SAN BENITO': '069',
    'SAN BERNARDINO': '071', 'SAN DIEGO': '073', 'SAN FRANCISCO': '075', 'SAN JOAQUIN': '077',
    'SAN LUIS OBISPO': '079', 'SAN MATEO': '081', 'SANTA BARBARA': '083', 'SANTA CLARA': '085',
    'SANTA CRUZ': '087', 'SHASTA': '089', 'SIERRA': '091', 'SISKIYOU': '093', 'SOLANO': '095',
    'SONOMA': '097', 'STANISLAUS': '099', 'SUTTER': '101', 'TEHAMA': '103', 'TRINITY': '105',
    'TULARE': '107', 'TUOLUMNE': '109', 'VENTURA': '111', 'YOLO': '113', 'YUBA': '115'
}

In [ ]:
# Build crosswalk from ACS data
crosswalk_clean = df_demo[['NAME', 'state', 'county']].copy()

# Create standardized county name
crosswalk_clean['county_name_raw'] = crosswalk_clean['NAME'].astype(str)
crosswalk_clean['county_name_clean'] = crosswalk_clean['county_name_raw'].apply(standardize_county_name)

# State FIPS for California is always "06"
crosswalk_clean['state_fips'] = "06"

# County FIPS from ACS data (already has 3-digit county codes)
crosswalk_clean['county_fips3'] = crosswalk_clean['county'].astype(str).str.zfill(3)

# Create fips5
crosswalk_clean['fips5'] = '06' + crosswalk_clean['county_fips3']

# Keep only needed columns
crosswalk_clean = crosswalk_clean[['county_name_raw', 'county_name_clean', 'county_fips3', 'state_fips', 'fips5']].copy()
crosswalk_clean = crosswalk_clean.dropna(subset=['fips5'])
crosswalk_clean = crosswalk_clean.drop_duplicates(subset=['fips5'])

print(f"\nCrosswalk created: {len(crosswalk_clean)} counties")
print(f"Unique fips5: {crosswalk_clean['fips5'].nunique()}")
print("\nSample:")
print(crosswalk_clean.head(10))

# Verify we have 58 CA counties
if crosswalk_clean['fips5'].nunique() < 58:
    print(f"\n⚠ WARNING: Only {crosswalk_clean['fips5'].nunique()} counties found. Expected 58.")
else:
    print(f"\n✓ Found {crosswalk_clean['fips5'].nunique()} counties (expected 58)")

# Save
crosswalk_clean.to_csv('outputs/data/county_crosswalk_clean.csv', index=False)
print("\n✓ Saved: outputs/data/county_crosswalk_clean.csv")

## 3) Clean Population (E4 estimates)

In [ ]:
# Process E4 population data
# The data is in wide format: County, Year1, Year2, etc.
# Header row is row 2, data starts row 3

print("Processing E4 population estimates...")
print(f"Raw shape: {df_e4_raw.shape}")
print(df_e4_raw.head(5))

# Find header row (row 2 has "County" and year labels)
header_row = 2
df_e4 = df_e4_raw.iloc[header_row:].copy()
df_e4.columns = df_e4_raw.iloc[header_row].values

# Clean column names
df_e4.columns = [str(c).strip() for c in df_e4.columns]
print(f"\nColumns after setting header: {df_e4.columns.tolist()}")

# Skip header row in data
df_e4 = df_e4.iloc[1:].copy()

# Get county column (first column)
county_col = df_e4.columns[0]
print(f"County column: {county_col}")

# Identify year columns (date formats like 1/1/2021)
year_cols = []
for col in df_e4.columns[1:]:
    try:
        # Try to extract year from column name
        if '/' in str(col):
            year = int(str(col).split('/')[-1])
            if 2000 <= year <= 2030:
                year_cols.append((col, year))
        elif str(col).isdigit() and len(str(col)) == 4:
            year_cols.append((col, int(col)))
    except:
        continue

print(f"Year columns found: {year_cols}")

# Melt to long format
if len(year_cols) > 0:
    # Create mapping of column names to years
    col_to_year = {col: year for col, year in year_cols}
    
    # Melt data
    id_vars = [county_col]
    value_vars = [col for col, year in year_cols]
    
    pop_long = pd.melt(df_e4, id_vars=id_vars, value_vars=value_vars, 
                       var_name='year_col', value_name='population')
    pop_long['year'] = pop_long['year_col'].map(col_to_year)
    pop_long['county_name_clean'] = pop_long[county_col].apply(standardize_county_name)
    
    # Merge with crosswalk to get FIPS
    pop_year = pop_long.merge(crosswalk_clean[['county_name_clean', 'fips5']], 
                              on='county_name_clean', how='left')
    
    # Clean population column
    pop_year['population'] = pd.to_numeric(pop_year['population'], errors='coerce')
    
    # Keep only needed columns
    pop_year = pop_year[['fips5', 'year', 'population']].copy()
    pop_year = pop_year.dropna(subset=['fips5', 'year', 'population'])
else:
    print("ERROR: Could not identify year columns")
    pop_year = pd.DataFrame(columns=['fips5', 'year', 'population'])

print(f"\nPopulation data: {len(pop_year)} county-years")
if len(pop_year) > 0:
    print(f"Year range: {pop_year['year'].min():.0f} - {pop_year['year'].max():.0f}")
    print(f"Unique counties: {pop_year['fips5'].nunique()}")
    print("\nSample:")
    print(pop_year.head(10))
else:
    print("WARNING: No population data loaded!")

# Save
pop_year.to_csv('outputs/data/pop_e4_clean.csv', index=False)
print("\n✓ Saved: outputs/data/pop_e4_clean.csv")

## 4) Clean Medi-Cal Enrollment

In [ ]:
# Inspect Medi-Cal enrollment structure
print("Medi-Cal enrollment columns:", df_medical_dash.columns.tolist())
print("\nFirst few rows:")
print(df_medical_dash.head(10))

# Identify key columns
date_cols = [c for c in df_medical_dash.columns if 'date' in c.lower() or 'eligibility' in c.lower()]
county_cols = [c for c in df_medical_dash.columns if 'county' in c.lower()]
enroll_cols = [c for c in df_medical_dash.columns if 'enroll' in c.lower() or '#' in c.lower()]

print(f"\nPotential date columns: {date_cols}")
print(f"Potential county columns: {county_cols}")
print(f"Potential enrollment columns: {enroll_cols}")

# Build enrollment dataset
medical_raw = df_medical_dash.copy()

# Extract county and standardize
if len(county_cols) > 0:
    county_col = county_cols[0]
    medical_raw['county_name_clean'] = medical_raw[county_col].apply(standardize_county_name)
    medical_raw = medical_raw.merge(crosswalk_clean[['county_name_clean', 'fips5']], on='county_name_clean', how='left')
else:
    print("ERROR: No county column found")
    medical_raw['fips5'] = None

# Extract date and year
if len(date_cols) > 0:
    date_col = date_cols[0]
    medical_raw['date'] = pd.to_datetime(medical_raw[date_col], errors='coerce')
    medical_raw['year'] = medical_raw['date'].dt.year
    medical_raw['month'] = medical_raw['date'].dt.month
else:
    print("WARNING: No date column found")
    medical_raw['year'] = None
    medical_raw['month'] = None

# Extract enrollment count
if len(enroll_cols) > 0:
    enroll_col = enroll_cols[0]
    # Handle NULL values
    medical_raw['enrollment'] = pd.to_numeric(medical_raw[enroll_col].replace('NULL', np.nan), errors='coerce')
else:
    print("ERROR: No enrollment column found")
    medical_raw['enrollment'] = np.nan

# The data is at county-year-sex-age-ethnic-language level
# First SUM across subgroups to get county-month total, then average across months
# Step 1: Sum enrollment within each county-year-month
medical_monthly = medical_raw.groupby(['fips5', 'year', 'month']).agg({
    'enrollment': 'sum'  # Sum across all demographic subgroups
}).reset_index()

# Step 2: Average monthly enrollment within each year
medical_year = medical_monthly.groupby(['fips5', 'year']).agg({
    'enrollment': 'mean'  # Average monthly enrollment
}).reset_index()
medical_year = medical_year.rename(columns={'enrollment': 'medi_cal_enrollment'})
medical_year = medical_year.dropna(subset=['fips5', 'year', 'medi_cal_enrollment'])

print(f"\nMedi-Cal enrollment: {len(medical_year)} county-years")
print(f"Year range: {medical_year['year'].min():.0f} - {medical_year['year'].max():.0f}")
print(f"Unique counties: {medical_year['fips5'].nunique()}")
print("\nSample:")
print(medical_year.head(10))

# Merge with population to compute share
medical_year = medical_year.merge(pop_year[['fips5', 'year', 'population']], on=['fips5', 'year'], how='left')
medical_year['medi_cal_share'] = medical_year['medi_cal_enrollment'] / medical_year['population']

print(f"\nAfter population merge: {len(medical_year)} county-years")
print(f"Medi-Cal share range: {medical_year['medi_cal_share'].min():.4f} - {medical_year['medi_cal_share'].max():.4f}")

# Save
medical_year.to_csv('outputs/data/medical_enrollment_clean.csv', index=False)
print("\n✓ Saved: outputs/data/medical_enrollment_clean.csv")

## 5) Clean Physician Supply

In [ ]:
# Define primary care specialties
# The pqi-physicians-specialities-list.xlsx maps conditions to specialties, not primary care definitions
# Using standard primary care specialty definitions

print("Defining primary care specialties...")

# Standard primary care specialties (based on AHRQ and common definitions)
# Do NOT include subspecialties
primary_care_specialties = {
    'FAMILY MEDICINE', 
    'FAMILY PRACTICE',
    'GENERAL PRACTICE',
    'GENERAL INTERNAL MEDICINE', 
    'INTERNAL MEDICINE',
    'GENERAL PEDIATRICS', 
    'PEDIATRICS',
    'GERIATRIC MEDICINE'
}

# Check what specialties are actually in the physician data
print("\nSpecialties in physician data:")
actual_specialties = df_physician['Specialty'].str.upper().unique()
print(actual_specialties[:20])

# Match primary care specialties to what's actually in the data
matched_specialties = set()
for spec in actual_specialties:
    spec_upper = str(spec).upper()
    if any(pcp in spec_upper for pcp in ['FAMILY', 'INTERNAL MEDICINE', 'PEDIATRIC', 'GENERAL PRACTICE', 'GERIATRIC']):
        # Exclude subspecialties
        if not any(sub in spec_upper for sub in ['CARDIO', 'GASTRO', 'NEPHRO', 'PULMON', 'ONCOL', 'HEMATOL', 'INFECTIOUS', 'RHEUMAT', 'ENDOCRIN', 'CRITICAL']):
            matched_specialties.add(spec_upper)

# Use matched specialties if found, otherwise use defaults
if len(matched_specialties) > 0:
    primary_care_specialties = matched_specialties
    print(f"\nMatched primary care specialties from data ({len(primary_care_specialties)}):")
else:
    print(f"\nUsing default primary care specialties ({len(primary_care_specialties)}):")

for spec in sorted(primary_care_specialties):
    print(f"  - {spec}")

# Save specialty definition
spec_df = pd.DataFrame({'specialty': sorted(primary_care_specialties)})
spec_df.to_csv('outputs/tables/primary_care_specialty_definition.csv', index=False)
print("\n✓ Saved: outputs/tables/primary_care_specialty_definition.csv")

In [ ]:
# Process physician supply data
print("Physician supply columns:", df_physician.columns.tolist())
print("\nFirst few rows:")
print(df_physician.head(10))

# Build physician supply dataset
physician_supply = df_physician.copy()

# Extract county and match to FIPS
physician_supply['county_name_clean'] = physician_supply['County'].apply(standardize_county_name)
physician_supply = physician_supply.merge(crosswalk_clean[['county_name_clean', 'fips5']], on='county_name_clean', how='left')

# No year column - data is time-invariant
print("NOTE: No year column found - treating as time-invariant")
physician_supply['year'] = None
IS_TIME_VARYING = False

# Extract specialty  
physician_supply['specialty_clean'] = physician_supply['Specialty'].astype(str).str.upper()

# Extract count - use 'Estimated Count' column
physician_supply['physician_count'] = pd.to_numeric(physician_supply['Estimated Count'], errors='coerce')

print(f"\nTotal physician records: {len(physician_supply)}")
print(f"Unique specialties: {physician_supply['specialty_clean'].nunique()}")
print(f"Unique counties: {physician_supply['fips5'].nunique()}")

# Filter to primary care specialties
physician_supply['is_primary_care'] = physician_supply['specialty_clean'].isin(primary_care_specialties)
physician_pcp = physician_supply[physician_supply['is_primary_care']].copy()

print(f"\nPrimary care physicians: {len(physician_pcp)} records")
print(f"PCP unique counties: {physician_pcp['fips5'].nunique()}")
print(f"Total PCP count: {physician_pcp['physician_count'].sum():.0f}")

# Aggregate to county level (sum all PCPs across activity levels)
physician_agg = physician_pcp.groupby(['fips5']).agg({
    'physician_count': 'sum'
}).reset_index()
physician_agg = physician_agg.rename(columns={'physician_count': 'pcp_supply'})

print(f"\nAggregated to {len(physician_agg)} counties")
print(f"PCP supply range: {physician_agg['pcp_supply'].min():.0f} - {physician_agg['pcp_supply'].max():.0f}")

# Merge with average population to compute per 100k
pop_avg = pop_year.groupby('fips5')['population'].mean().reset_index()
pop_avg = pop_avg.rename(columns={'population': 'avg_population'})
physician_agg = physician_agg.merge(pop_avg, on='fips5', how='left')

print(f"\nAfter population merge: {len(physician_agg)} counties")
print(f"Population range: {physician_agg['avg_population'].min():.0f} - {physician_agg['avg_population'].max():.0f}")

# Compute per 100k
physician_agg['pcp_per_100k'] = (physician_agg['pcp_supply'] / physician_agg['avg_population']) * 100000

print(f"PCP per 100k range: {physician_agg['pcp_per_100k'].min():.2f} - {physician_agg['pcp_per_100k'].max():.2f}")

# Save
physician_agg.to_csv('outputs/data/physician_supply_clean.csv', index=False)
print("\n✓ Saved: outputs/data/physician_supply_clean.csv")

## 6) Clean Shortage File

In [ ]:
# Inspect shortage file structure
print("Shortage file columns:", df_shortage.columns.tolist())
print("\nFirst few rows:")
print(df_shortage.head(10))

# Identify key columns
county_cols = [c for c in df_shortage.columns if 'county' in c.lower() or 'cnty' in c.lower() or 'fips' in c.lower()]
year_cols = [c for c in df_shortage.columns if 'year' in c.lower() or 'effective' in c.lower()]
flag_cols = [c for c in df_shortage.columns if 'pcs' in c.lower() or 'shortage' in c.lower() or 'yes' in c.lower()]
score_cols = [c for c in df_shortage.columns if 'score' in c.lower()]

print(f"\nPotential county columns: {county_cols}")
print(f"Potential year columns: {year_cols}")
print(f"Potential flag columns: {flag_cols}")
print(f"Potential score columns: {score_cols}")

# Build shortage dataset
shortage_raw = df_shortage.copy()

# Extract county FIPS
if 'CNTY_FIPS' in shortage_raw.columns:
    shortage_raw['county_fips3'] = pd.to_numeric(shortage_raw['CNTY_FIPS'], errors='coerce').astype(str).str.zfill(3)
    shortage_raw['fips5'] = '06' + shortage_raw['county_fips3']
elif len(county_cols) > 0:
    county_col = county_cols[0]
    if 'FIPS' in county_col.upper():
        shortage_raw['county_fips3'] = pd.to_numeric(shortage_raw[county_col], errors='coerce').astype(str).str.zfill(3)
        shortage_raw['fips5'] = '06' + shortage_raw['county_fips3']
    else:
        shortage_raw['county_name_clean'] = shortage_raw[county_col].apply(standardize_county_name)
        shortage_raw = shortage_raw.merge(crosswalk_clean[['county_name_clean', 'fips5']], on='county_name_clean', how='left')
else:
    print("ERROR: Could not identify county column")
    shortage_raw['fips5'] = None

# Extract year (if exists)
if len(year_cols) > 0:
    year_col = year_cols[0]
    if 'effective' in year_col.lower():
        # Parse date
        shortage_raw['date'] = pd.to_datetime(shortage_raw[year_col], errors='coerce')
        shortage_raw['year'] = shortage_raw['date'].dt.year
    else:
        shortage_raw['year'] = pd.to_numeric(shortage_raw[year_col], errors='coerce')
else:
    print("NOTE: No year column found - treating as time-invariant")
    shortage_raw['year'] = None

# Extract shortage flag
# PCSA column indicates Primary Care Service Area shortage designation
if 'PCSA' in shortage_raw.columns:
    shortage_raw['shortage_flag'] = (shortage_raw['PCSA'].astype(str).str.upper() == 'YES').astype(int)
elif len(flag_cols) > 0:
    flag_col = flag_cols[0]
    shortage_raw['shortage_flag'] = (shortage_raw[flag_col].astype(str).str.upper().isin(['YES', 'TRUE', '1', 'Y'])).astype(int)
else:
    print("WARNING: Could not identify shortage flag column")
    shortage_raw['shortage_flag'] = 0

# Extract score (if available)
if len(score_cols) > 0:
    score_col = [c for c in score_cols if 'total' in c.lower() or 'tota' in c.lower()]
    if len(score_col) > 0:
        shortage_raw['shortage_score'] = pd.to_numeric(shortage_raw[score_col[0]], errors='coerce')
    else:
        shortage_raw['shortage_score'] = pd.to_numeric(shortage_raw[score_cols[0]], errors='coerce')
else:
    shortage_raw['shortage_score'] = None

# Aggregate to county-year (or county if no year)
# If sub-county (MSSA level), aggregate by max(flag) and max(score)
if shortage_raw['year'].notna().any():
    shortage_agg = shortage_raw.groupby(['fips5', 'year']).agg({
        'shortage_flag': 'max',
        'shortage_score': 'max'
    }).reset_index()
    IS_SHORTAGE_TIME_VARYING = True
else:
    shortage_agg = shortage_raw.groupby(['fips5']).agg({
        'shortage_flag': 'max',
        'shortage_score': 'max'
    }).reset_index()
    IS_SHORTAGE_TIME_VARYING = False
    print("\n⚠ WARNING: Shortage designation is time-invariant. Will merge by county only.")

shortage_agg = shortage_agg.dropna(subset=['fips5'])

print(f"\nShortage designation: {len(shortage_agg)} county{'s' if not IS_SHORTAGE_TIME_VARYING else '-years'}")
print(f"Counties with shortage: {(shortage_agg['shortage_flag'] == 1).sum()}")

# Save
shortage_agg.to_csv('outputs/data/shortage_clean.csv', index=False)
print("\n✓ Saved: outputs/data/shortage_clean.csv")

In [ ]:
# Clean demographic ACS
print("Cleaning demoACS.csv...")
acs_demo = df_demo.copy()

# Extract county name and standardize
acs_demo['county_name_clean'] = acs_demo['NAME'].apply(standardize_county_name)
acs_demo = acs_demo.merge(crosswalk_clean[['county_name_clean', 'fips5']], on='county_name_clean', how='left')

# Extract required variables
acs_demo['age65_pct'] = pd.to_numeric(acs_demo['DP05_0024PE'], errors='coerce')
acs_demo['hispanic_pct'] = pd.to_numeric(acs_demo['DP05_0071PE'], errors='coerce')

acs_demo_clean = acs_demo[['fips5', 'age65_pct', 'hispanic_pct']].copy()
acs_demo_clean = acs_demo_clean.dropna(subset=['fips5'])

print(f"  Counties: {acs_demo_clean['fips5'].nunique()}")

# Clean education ACS
print("\nCleaning educACS.csv...")
acs_educ = df_educ.copy()

acs_educ['county_name_clean'] = acs_educ['NAME'].apply(standardize_county_name)
acs_educ = acs_educ.merge(crosswalk_clean[['county_name_clean', 'fips5']], on='county_name_clean', how='left')

acs_educ['bachelors_pct'] = pd.to_numeric(acs_educ['DP02_0068PE'], errors='coerce')

acs_educ_clean = acs_educ[['fips5', 'bachelors_pct']].copy()
acs_educ_clean = acs_educ_clean.dropna(subset=['fips5'])

print(f"  Counties: {acs_educ_clean['fips5'].nunique()}")

# Clean economic ACS
print("\nCleaning EconACS.csv...")
acs_econ = df_econ.copy()

acs_econ['county_name_clean'] = acs_econ['NAME'].apply(standardize_county_name)
acs_econ = acs_econ.merge(crosswalk_clean[['county_name_clean', 'fips5']], on='county_name_clean', how='left')

# Use only poverty_pct and unemp_pct (ignore income field)
acs_econ['poverty_pct'] = pd.to_numeric(acs_econ['DP03_0128PE'], errors='coerce')
acs_econ['unemp_pct'] = pd.to_numeric(acs_econ['DP03_0009PE'], errors='coerce')

acs_econ_clean = acs_econ[['fips5', 'poverty_pct', 'unemp_pct']].copy()
acs_econ_clean = acs_econ_clean.dropna(subset=['fips5'])

print(f"  Counties: {acs_econ_clean['fips5'].nunique()}")

# Merge all ACS controls
acs_controls = acs_demo_clean.merge(acs_educ_clean, on='fips5', how='outer')
acs_controls = acs_controls.merge(acs_econ_clean, on='fips5', how='outer')

print(f"\nACS controls: {len(acs_controls)} counties")
print(f"Variables: {list(acs_controls.columns)}")
print("\nSample:")
print(acs_controls.head(10))

# Save
acs_controls.to_csv('outputs/data/acs_controls_clean.csv', index=False)
print("\n✓ Saved: outputs/data/acs_controls_clean.csv")

## 8) Clean PQI and Build Outcomes

In [ ]:
# Clean PQI data
print("Cleaning PQI.csv...")

pqi_clean = df_pqi.copy()

# Standardize county name
pqi_clean['county_name_clean'] = pqi_clean['County'].apply(standardize_county_name)
pqi_clean = pqi_clean.merge(crosswalk_clean[['county_name_clean', 'fips5']], on='county_name_clean', how='left')

# Extract year
pqi_clean['year'] = pd.to_numeric(pqi_clean['Year'], errors='coerce')

# Convert all numeric columns first (handle commas in numbers)
for col in ['Count_ICD9', 'Count_ICD10', 'Population_ICD9', 'Population_ICD10', 
            'ObsRate_ICD9', 'ObsRate_ICD10', 'RiskAdjRate_ICD9', 'RiskAdjRate_ICD10']:
    if col in pqi_clean.columns:
        pqi_clean[col] = pqi_clean[col].astype(str).str.replace(',', '').replace('nan', '')
        pqi_clean[col] = pd.to_numeric(pqi_clean[col], errors='coerce')

# Unify ICD-9 and ICD-10 fields
# Use ICD-10 if not null, else ICD-9
pqi_clean['pqi_count'] = pqi_clean['Count_ICD10'].fillna(pqi_clean['Count_ICD9'])
pqi_clean['pqi_population'] = pqi_clean['Population_ICD10'].fillna(pqi_clean['Population_ICD9'])
pqi_clean['obs_rate'] = pqi_clean['ObsRate_ICD10'].fillna(pqi_clean['ObsRate_ICD9'])
pqi_clean['risk_adj_rate'] = pqi_clean['RiskAdjRate_ICD10'].fillna(pqi_clean['RiskAdjRate_ICD9'])

# Main outcome: risk_adj_rate if available, else obs_rate
pqi_clean['outcome_rate'] = pqi_clean['risk_adj_rate'].fillna(pqi_clean['obs_rate'])

# Ensure numeric types
pqi_clean['outcome_rate'] = pd.to_numeric(pqi_clean['outcome_rate'], errors='coerce')
pqi_clean['pqi_count'] = pd.to_numeric(pqi_clean['pqi_count'], errors='coerce')
pqi_clean['pqi_population'] = pd.to_numeric(pqi_clean['pqi_population'], errors='coerce')

# Keep PQI identifier
pqi_clean['pqi_id'] = pqi_clean['PQI']
pqi_clean['pqi_name'] = pqi_clean['PQIDescription']

# Create long format
pqi_long_clean = pqi_clean[[
    'fips5', 'year', 'pqi_id', 'pqi_name', 
    'outcome_rate', 'pqi_count', 'pqi_population'
]].copy()
pqi_long_clean = pqi_long_clean.dropna(subset=['fips5', 'year', 'outcome_rate'])

print(f"PQI long format: {len(pqi_long_clean)} county-year-PQI observations")
print(f"Year range: {pqi_long_clean['year'].min():.0f} - {pqi_long_clean['year'].max():.0f}")
print(f"Unique counties: {pqi_long_clean['fips5'].nunique()}")
print(f"Unique PQI measures: {pqi_long_clean['pqi_id'].nunique()}")
print(f"Outcome rate range: {pqi_long_clean['outcome_rate'].min():.2f} - {pqi_long_clean['outcome_rate'].max():.2f}")

# Save long format
pqi_long_clean.to_csv('outputs/data/pqi_long_clean.csv', index=False)
print("\n✓ Saved: outputs/data/pqi_long_clean.csv")

In [ ]:
# Aggregate to county-year (mean across all PQIs)
pqi_county_year = pqi_long_clean.groupby(['fips5', 'year']).agg({
    'outcome_rate': 'mean',
    'pqi_count': 'sum',
    'pqi_population': 'mean'  # Should be same across PQIs for same county-year
}).reset_index()
pqi_county_year = pqi_county_year.rename(columns={
    'outcome_rate': 'pqi_mean_rate',
    'pqi_count': 'pqi_sum_count',
    'pqi_population': 'pqi_sum_population'
})

print(f"PQI county-year: {len(pqi_county_year)} county-years")
print(f"Year range: {pqi_county_year['year'].min():.0f} - {pqi_county_year['year'].max():.0f}")
print(f"Unique counties: {pqi_county_year['fips5'].nunique()}")
print(f"PQI mean rate range: {pqi_county_year['pqi_mean_rate'].min():.2f} - {pqi_county_year['pqi_mean_rate'].max():.2f}")

# Save
pqi_county_year.to_csv('outputs/data/pqi_county_year_clean.csv', index=False)
print("\n✓ Saved: outputs/data/pqi_county_year_clean.csv")

In [ ]:
# Merge plan
print("=" * 60)
print("MERGE PLAN")
print("=" * 60)
print("\nStarting with: pqi_county_year (fips5, year)")
print("Merge keys and intended final schema:")
print("  1. + pop_year (fips5, year) -> population")
print("  2. + medical_year (fips5, year) -> medi_cal_enrollment, medi_cal_share")
print("  3. + physician_agg (fips5, year or fips5) -> pcp_supply, pcp_per_100k")
print("  4. + shortage_agg (fips5, year or fips5) -> shortage_flag, shortage_score")
print("  5. + acs_controls (fips5) -> poverty_pct, unemp_pct, age65_pct, hispanic_pct, bachelors_pct")
print("\nFinal panel: county-year with all variables")
print("=" * 60)

In [ ]:
# Start with PQI outcomes
master_panel = pqi_county_year.copy()

print(f"Starting rows: {len(master_panel)}")
print(f"Unique counties: {master_panel['fips5'].nunique()}")
print(f"Year range: {master_panel['year'].min():.0f} - {master_panel['year'].max():.0f}")

# Merge population
master_panel = merge_report(master_panel, pop_year, on=['fips5', 'year'], 
                           left_name='PQI', right_name='Population')
master_panel = master_panel.drop(columns=['_merge'])

# Merge Medi-Cal enrollment
master_panel = merge_report(master_panel, medical_year[['fips5', 'year', 'medi_cal_enrollment', 'medi_cal_share']], 
                           on=['fips5', 'year'], left_name='Panel', right_name='Medi-Cal')
master_panel = master_panel.drop(columns=['_merge'])

# Merge physician supply
if IS_TIME_VARYING:
    master_panel = merge_report(master_panel, physician_agg[['fips5', 'year', 'pcp_supply', 'pcp_per_100k']], 
                               on=['fips5', 'year'], left_name='Panel', right_name='Physician Supply')
else:
    master_panel = merge_report(master_panel, physician_agg[['fips5', 'pcp_supply', 'pcp_per_100k']], 
                               on=['fips5'], left_name='Panel', right_name='Physician Supply')
master_panel = master_panel.drop(columns=['_merge'])

# Merge shortage
if IS_SHORTAGE_TIME_VARYING:
    master_panel = merge_report(master_panel, shortage_agg[['fips5', 'year', 'shortage_flag', 'shortage_score']], 
                               on=['fips5', 'year'], left_name='Panel', right_name='Shortage')
else:
    master_panel = merge_report(master_panel, shortage_agg[['fips5', 'shortage_flag', 'shortage_score']], 
                               on=['fips5'], left_name='Panel', right_name='Shortage')
master_panel = master_panel.drop(columns=['_merge'])

# Merge ACS controls (time-invariant)
master_panel = merge_report(master_panel, acs_controls, on=['fips5'], 
                           left_name='Panel', right_name='ACS Controls')
master_panel = master_panel.drop(columns=['_merge'])

print(f"\nFinal panel: {len(master_panel)} county-years")
print(f"Unique counties: {master_panel['fips5'].nunique()}")
print(f"Year range: {master_panel['year'].min():.0f} - {master_panel['year'].max():.0f}")

# QA: Check missingness
print("\n" + "=" * 60)
print("MISSINGNESS REPORT")
print("=" * 60)
summarize_missingness(master_panel, "Master Panel")

In [ ]:
# Create desert definitions
print("\n" + "=" * 60)
print("CREATING DESERT DEFINITIONS")
print("=" * 60)

# Define thresholds and quartiles for each year
desert_panel = master_panel.copy()

# For each year, compute quartiles and thresholds
desert_panel['high_medi_cal_q'] = False
desert_panel['low_pcp_q'] = False
desert_panel['high_medi_cal_thr'] = False
desert_panel['low_pcp_thr'] = False

# Thresholds
medi_cal_threshold = 0.30  # 30% Medi-Cal share
# PCP threshold: choose based on distribution (e.g., bottom 25th percentile value)
pcp_threshold = desert_panel['pcp_per_100k'].quantile(0.25)

print(f"Medi-Cal share threshold: {medi_cal_threshold}")
print(f"PCP per 100k threshold: {pcp_threshold:.2f}")

for year in desert_panel['year'].unique():
    year_mask = desert_panel['year'] == year
    year_data = desert_panel[year_mask]
    
    # Quartile-based
    medi_cal_q75 = year_data['medi_cal_share'].quantile(0.75)
    pcp_q25 = year_data['pcp_per_100k'].quantile(0.25)
    
    desert_panel.loc[year_mask, 'high_medi_cal_q'] = desert_panel.loc[year_mask, 'medi_cal_share'] >= medi_cal_q75
    desert_panel.loc[year_mask, 'low_pcp_q'] = desert_panel.loc[year_mask, 'pcp_per_100k'] <= pcp_q25
    
    # Threshold-based
    desert_panel.loc[year_mask, 'high_medi_cal_thr'] = desert_panel.loc[year_mask, 'medi_cal_share'] >= medi_cal_threshold
    desert_panel.loc[year_mask, 'low_pcp_thr'] = desert_panel.loc[year_mask, 'pcp_per_100k'] <= pcp_threshold

# Create desert definitions
# Definition 1: high Medi-Cal AND low PCP
desert_panel['desert_q_def1'] = (desert_panel['high_medi_cal_q'] & desert_panel['low_pcp_q']).astype(int)
desert_panel['desert_thr_def1'] = (desert_panel['high_medi_cal_thr'] & desert_panel['low_pcp_thr']).astype(int)

# Definition 2: high Medi-Cal AND (low PCP OR shortage)
desert_panel['desert_q_def2'] = (desert_panel['high_medi_cal_q'] & 
                                 (desert_panel['low_pcp_q'] | (desert_panel['shortage_flag'] == 1))).astype(int)
desert_panel['desert_thr_def2'] = (desert_panel['high_medi_cal_thr'] & 
                                   (desert_panel['low_pcp_thr'] | (desert_panel['shortage_flag'] == 1))).astype(int)

print(f"\nDesert counts (quartile-based, def1): {desert_panel['desert_q_def1'].sum()}")
print(f"Desert counts (quartile-based, def2): {desert_panel['desert_q_def2'].sum()}")
print(f"Desert counts (threshold-based, def1): {desert_panel['desert_thr_def1'].sum()}")
print(f"Desert counts (threshold-based, def2): {desert_panel['desert_thr_def2'].sum()}")

# Update master panel
master_panel = desert_panel.copy()

# Save master panel
master_panel.to_csv('outputs/data/ca_master_panel.csv', index=False)
print("\n✓ Saved: outputs/data/ca_master_panel.csv")

In [ ]:
# Create variable dictionary
var_dict = pd.DataFrame([
    ['fips5', '5-digit FIPS code (state + county)', 'county name.xlsx', 'code', 'No'],
    ['year', 'Year', 'PQI.csv', 'year', 'Yes'],
    ['pqi_mean_rate', 'Mean PQI rate (risk-adjusted if available, else observed)', 'PQI.csv', 'rate per 100k', 'Yes'],
    ['pqi_sum_count', 'Total PQI counts', 'PQI.csv', 'count', 'Yes'],
    ['pqi_sum_population', 'Population denominator for PQI', 'PQI.csv', 'count', 'Yes'],
    ['population', 'County population', 'E4 estiamtes.xlsx', 'count', 'Yes'],
    ['medi_cal_enrollment', 'Medi-Cal enrollment (annual average monthly)', 'medi-cal-enrollment-dashboard-data.csv', 'count', 'Yes'],
    ['medi_cal_share', 'Medi-Cal enrollment / population', 'medi-cal-enrollment-dashboard-data.csv', 'proportion', 'Yes'],
    ['pcp_supply', 'Primary care physician supply (count)', 'physicians-actively-working-by-specialty-and-patient-care-hours.xlsx', 'count', 'Yes' if IS_TIME_VARYING else 'No'],
    ['pcp_per_100k', 'Primary care physicians per 100,000 population', 'physicians-actively-working-by-specialty-and-patient-care-hours.xlsx', 'rate', 'Yes' if IS_TIME_VARYING else 'No'],
    ['shortage_flag', 'Primary care shortage designation (0/1)', 'Primary CAre Shortage .csv', 'binary', 'Yes' if IS_SHORTAGE_TIME_VARYING else 'No'],
    ['shortage_score', 'Primary care shortage score', 'Primary CAre Shortage .csv', 'score', 'Yes' if IS_SHORTAGE_TIME_VARYING else 'No'],
    ['poverty_pct', 'Percent below poverty line', 'EconACS.csv', 'percent', 'No'],
    ['unemp_pct', 'Unemployment rate', 'EconACS.csv', 'percent', 'No'],
    ['age65_pct', 'Percent age 65 and over', 'demoACS.csv', 'percent', 'No'],
    ['hispanic_pct', 'Percent Hispanic', 'demoACS.csv', 'percent', 'No'],
    ['bachelors_pct', 'Percent with bachelor degree or higher', 'educACS.csv', 'percent', 'No'],
    ['desert_q_def1', 'Desert (quartile): high Medi-Cal & low PCP', 'Derived', 'binary', 'Yes'],
    ['desert_q_def2', 'Desert (quartile): high Medi-Cal & (low PCP OR shortage)', 'Derived', 'binary', 'Yes'],
    ['desert_thr_def1', 'Desert (threshold): high Medi-Cal & low PCP', 'Derived', 'binary', 'Yes'],
    ['desert_thr_def2', 'Desert (threshold): high Medi-Cal & (low PCP OR shortage)', 'Derived', 'binary', 'Yes'],
], columns=['Variable', 'Description', 'Source File', 'Unit', 'Time-Varying'])

var_dict.to_csv('outputs/data/ca_variable_dictionary.csv', index=False)
print("✓ Saved: outputs/data/ca_variable_dictionary.csv")
print("\nVariable dictionary:")
print(var_dict.to_string())

## 10) Descriptive Figures

In [ ]:
# Set matplotlib style
plt.style.use('default')
fig_dpi = 300

# 1. Histogram of medi_cal_share with threshold
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(master_panel['medi_cal_share'].dropna(), bins=50, edgecolor='black', alpha=0.7)
ax.axvline(medi_cal_threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {medi_cal_threshold}')
ax.set_xlabel('Medi-Cal Share', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of Medi-Cal Share Across County-Years', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/figures/hist_medi_cal_share.png', dpi=fig_dpi, bbox_inches='tight')
print("✓ Saved: outputs/figures/hist_medi_cal_share.png")
plt.close()

In [ ]:
# 2. Histogram of pcp_per_100k
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(master_panel['pcp_per_100k'].dropna(), bins=50, edgecolor='black', alpha=0.7)
ax.axvline(pcp_threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {pcp_threshold:.2f}')
ax.set_xlabel('Primary Care Physicians per 100,000', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of Primary Care Physician Supply (per 100k)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/figures/hist_pcp_per_100k.png', dpi=fig_dpi, bbox_inches='tight')
print("✓ Saved: outputs/figures/hist_pcp_per_100k.png")
plt.close()

In [ ]:
# 3. Time series: statewide (population-weighted) medi_cal_share and pqi_mean_rate
statewide = master_panel.groupby('year').apply(
    lambda x: pd.Series({
        'medi_cal_share_weighted': np.average(x['medi_cal_share'], weights=x['population']),
        'pqi_mean_rate_weighted': np.average(x['pqi_mean_rate'], weights=x['population'])
    })
).reset_index()

fig, ax1 = plt.subplots(figsize=(12, 6))
ax2 = ax1.twinx()

line1 = ax1.plot(statewide['year'], statewide['medi_cal_share_weighted'], 
                'b-', linewidth=2, marker='o', label='Medi-Cal Share')
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Medi-Cal Share (Population-Weighted)', fontsize=12, color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.grid(alpha=0.3)

line2 = ax2.plot(statewide['year'], statewide['pqi_mean_rate_weighted'], 
                'r-', linewidth=2, marker='s', label='PQI Mean Rate')
ax2.set_ylabel('PQI Mean Rate (Population-Weighted)', fontsize=12, color='r')
ax2.tick_params(axis='y', labelcolor='r')

ax1.set_title('Statewide Trends: Medi-Cal Share and PQI Rates', fontsize=14, fontweight='bold')
fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))
plt.tight_layout()
plt.savefig('outputs/figures/time_series_statewide.png', dpi=fig_dpi, bbox_inches='tight')
print("✓ Saved: outputs/figures/time_series_statewide.png")
plt.close()

In [ ]:
# 4. Scatter: medi_cal_share vs pcp_per_100k (label top 5 outliers)
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(master_panel['medi_cal_share'], master_panel['pcp_per_100k'], 
          alpha=0.5, s=30, edgecolors='black', linewidth=0.5)

# Identify top 5 outliers (by distance from center)
center_x = master_panel['medi_cal_share'].median()
center_y = master_panel['pcp_per_100k'].median()
master_panel['dist_from_center'] = np.sqrt(
    ((master_panel['medi_cal_share'] - center_x) ** 2) + 
    ((master_panel['pcp_per_100k'] - center_y) ** 2)
)
top_outliers = master_panel.nlargest(5, 'dist_from_center')

# Label outliers
for idx, row in top_outliers.iterrows():
    county_name = crosswalk_clean[crosswalk_clean['fips5'] == row['fips5']]['county_name_clean'].values
    if len(county_name) > 0:
        ax.annotate(county_name[0], (row['medi_cal_share'], row['pcp_per_100k']), 
                   fontsize=8, alpha=0.7)

ax.set_xlabel('Medi-Cal Share', fontsize=12)
ax.set_ylabel('Primary Care Physicians per 100,000', fontsize=12)
ax.set_title('Medi-Cal Share vs Primary Care Supply', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/figures/scatter_medi_cal_vs_pcp.png', dpi=fig_dpi, bbox_inches='tight')
print("✓ Saved: outputs/figures/scatter_medi_cal_vs_pcp.png")
plt.close()

In [ ]:
# 5. Scatter: pcp_per_100k vs pqi_mean_rate (with fitted line)
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(master_panel['pcp_per_100k'], master_panel['pqi_mean_rate'], 
          alpha=0.5, s=30, edgecolors='black', linewidth=0.5)

# Fit line
mask = master_panel[['pcp_per_100k', 'pqi_mean_rate']].notna().all(axis=1)
if mask.sum() > 0:
    x_fit = master_panel.loc[mask, 'pcp_per_100k']
    y_fit = master_panel.loc[mask, 'pqi_mean_rate']
    z = np.polyfit(x_fit, y_fit, 1)
    p = np.poly1d(z)
    ax.plot(x_fit, p(x_fit), "r--", alpha=0.8, linewidth=2, label=f'Fitted line: y={z[0]:.2f}x+{z[1]:.2f}')
    ax.legend()

ax.set_xlabel('Primary Care Physicians per 100,000', fontsize=12)
ax.set_ylabel('PQI Mean Rate', fontsize=12)
ax.set_title('Primary Care Supply vs Preventable Hospitalizations', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/figures/scatter_pcp_vs_pqi.png', dpi=fig_dpi, bbox_inches='tight')
print("✓ Saved: outputs/figures/scatter_pcp_vs_pqi.png")
plt.close()

In [ ]:
# 6. Bar: top/bottom 10 counties in pqi_mean_rate for latest year
latest_year = master_panel['year'].max()
latest_data = master_panel[master_panel['year'] == latest_year].copy()
latest_data = latest_data.merge(crosswalk_clean[['fips5', 'county_name_clean']], on='fips5', how='left')

top10 = latest_data.nlargest(10, 'pqi_mean_rate')
bottom10 = latest_data.nsmallest(10, 'pqi_mean_rate')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Top 10
ax1.barh(range(len(top10)), top10['pqi_mean_rate'], color='coral', edgecolor='black')
ax1.set_yticks(range(len(top10)))
ax1.set_yticklabels(top10['county_name_clean'], fontsize=9)
ax1.set_xlabel('PQI Mean Rate', fontsize=12)
ax1.set_title(f'Top 10 Counties: Highest PQI Rates ({latest_year:.0f})', fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
ax1.invert_yaxis()

# Bottom 10
ax2.barh(range(len(bottom10)), bottom10['pqi_mean_rate'], color='lightblue', edgecolor='black')
ax2.set_yticks(range(len(bottom10)))
ax2.set_yticklabels(bottom10['county_name_clean'], fontsize=9)
ax2.set_xlabel('PQI Mean Rate', fontsize=12)
ax2.set_title(f'Bottom 10 Counties: Lowest PQI Rates ({latest_year:.0f})', fontsize=12, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)
ax2.invert_yaxis()

plt.tight_layout()
plt.savefig('outputs/figures/bar_top_bottom_pqi.png', dpi=fig_dpi, bbox_inches='tight')
print("✓ Saved: outputs/figures/bar_top_bottom_pqi.png")
plt.close()

In [ ]:
# Optional maps (only if shapefile exists)
# Check for common shapefile names
shapefile_candidates = [
    'ca_counties.shp', 'california_counties.shp', 'CA_Counties.shp',
    'counties.shp', 'ca_county_boundaries.shp'
]

shapefile_path = None
for candidate in shapefile_candidates:
    if os.path.exists(candidate):
        shapefile_path = candidate
        break

if shapefile_path and HAS_GEOPANDAS:
    print(f"Found shapefile: {shapefile_path}")
    try:
        gdf = gpd.read_file(shapefile_path)
        # Ensure FIPS column exists and matches
        if 'FIPS' in gdf.columns or 'GEOID' in gdf.columns:
            fips_col = 'FIPS' if 'FIPS' in gdf.columns else 'GEOID'
            gdf['fips5'] = gdf[fips_col].astype(str).str.zfill(5)
            
            # Choose a year for mapping
            map_year = master_panel['year'].median()
            map_data = master_panel[master_panel['year'] == map_year].copy()
            
            # Merge
            gdf_map = gdf.merge(map_data, on='fips5', how='left')
            
            # Create maps
            fig, axes = plt.subplots(2, 2, figsize=(16, 16))
            axes = axes.flatten()
            
            vars_to_map = ['medi_cal_share', 'pcp_per_100k', 'pqi_mean_rate', 'desert_q_def2']
            titles = ['Medi-Cal Share', 'PCP per 100k', 'PQI Mean Rate', 'Desert (Quartile Def2)']
            
            for ax, var, title in zip(axes, vars_to_map, titles):
                if var in gdf_map.columns:
                    gdf_map.plot(column=var, ax=ax, legend=True, cmap='YlOrRd', 
                               missing_kwds={'color': 'lightgrey'}, edgecolor='black', linewidth=0.5)
                    ax.set_title(f'{title} ({map_year:.0f})', fontsize=12, fontweight='bold')
                    ax.axis('off')
            
            plt.tight_layout()
            plt.savefig('outputs/figures/maps_california.png', dpi=fig_dpi, bbox_inches='tight')
            print("✓ Saved: outputs/figures/maps_california.png")
            plt.close()
    except Exception as e:
        print(f"⚠ Could not create maps: {e}")
else:
    print("⚠ No shapefile found or geopandas not available. Skipping maps.")
    print("  (This is expected - maps are optional per requirements)")

## 11) Regression Analyses

In [ ]:
# Prepare data for regression
reg_data = master_panel.copy()

# Drop rows with missing key variables
key_vars = ['pqi_mean_rate', 'medi_cal_share', 'pcp_per_100k', 'shortage_flag',
           'poverty_pct', 'unemp_pct', 'age65_pct', 'hispanic_pct', 'bachelors_pct']
reg_data = reg_data.dropna(subset=key_vars)

print(f"Regression sample: {len(reg_data)} county-years")
print(f"Unique counties: {reg_data['fips5'].nunique()}")
print(f"Year range: {reg_data['year'].min():.0f} - {reg_data['year'].max():.0f}")

# Set index for PanelOLS if using
if HAS_PANELOLS:
    reg_data_indexed = reg_data.set_index(['fips5', 'year']).sort_index()
else:
    reg_data_indexed = reg_data.copy()

# Prepare controls
controls = ['poverty_pct', 'unemp_pct', 'age65_pct', 'hispanic_pct', 'bachelors_pct']

In [ ]:
# MODEL A: Provider Supply Model
print("=" * 60)
print("MODEL A: Provider Supply Model")
print("=" * 60)

if IS_TIME_VARYING:
    print("PCP supply is time-varying. Using panel model.")
    Y1 = reg_data_indexed['pcp_per_100k']
    X1 = reg_data_indexed[['medi_cal_share', 'shortage_flag'] + controls]
    
    if HAS_PANELOLS:
        model_a = PanelOLS(Y1, X1, entity_effects=True, time_effects=True)
        results_a = model_a.fit(cov_type='clustered', cluster_entity=True)
    else:
        # Use statsmodels with dummies
        X1_with_dummies = X1.copy()
        X1_with_dummies = pd.get_dummies(X1_with_dummies, columns=['fips5'], prefix='county', drop_first=False)
        X1_with_dummies = pd.get_dummies(X1_with_dummies, columns=['year'], prefix='year', drop_first=False)
        X1_with_dummies = sm.add_constant(X1_with_dummies)
        model_a = OLS(Y1, X1_with_dummies)
        results_a = model_a.fit(cov_type='cluster', cov_kwds={'groups': reg_data_indexed.index.get_level_values(0)})
else:
    print("PCP supply is time-invariant. Using cross-sectional model.")
    # Aggregate to county level
    reg_data_cs = reg_data.groupby('fips5').agg({
        'pcp_per_100k': 'first',
        'medi_cal_share': 'mean',
        'shortage_flag': 'first',
        'poverty_pct': 'first',
        'unemp_pct': 'first',
        'age65_pct': 'first',
        'hispanic_pct': 'first',
        'bachelors_pct': 'first'
    }).reset_index()
    
    Y1 = reg_data_cs['pcp_per_100k']
    X1 = reg_data_cs[['medi_cal_share', 'shortage_flag'] + controls]
    X1 = sm.add_constant(X1)
    
    model_a = OLS(Y1, X1)
    results_a = model_a.fit()

print(results_a.summary())

# Save results
results_a_df = pd.DataFrame({
    'Variable': results_a.params.index,
    'Coefficient': results_a.params.values,
    'Std_Error': results_a.std_errors.values if hasattr(results_a, 'std_errors') else results_a.bse.values,
    'P_Value': results_a.pvalues.values,
    'CI_Lower': results_a.conf_int()[0].values,
    'CI_Upper': results_a.conf_int()[1].values
})
results_a_df.to_csv('outputs/tables/reg_model_a_supply.csv', index=False)
print("\n✓ Saved: outputs/tables/reg_model_a_supply.csv")

In [ ]:
# MODEL B: Outcomes Model
print("\n" + "=" * 60)
print("MODEL B: Outcomes Model")
print("=" * 60)

X2_vars = ['pcp_per_100k', 'medi_cal_share', 'shortage_flag'] + controls

if HAS_PANELOLS:
    Y2 = reg_data_indexed['pqi_mean_rate']
    X2 = reg_data_indexed[X2_vars]
    model_b = PanelOLS(Y2, X2, entity_effects=True, time_effects=True)
    results_b = model_b.fit(cov_type='clustered', cluster_entity=True)
else:
    # Use simple OLS with controls (no fixed effects since data is limited)
    Y2 = reg_data['pqi_mean_rate']
    X2 = reg_data[X2_vars].copy()
    X2 = sm.add_constant(X2)
    model_b = OLS(Y2, X2)
    results_b = model_b.fit()

print(results_b.summary())

# Save results
results_b_df = pd.DataFrame({
    'Variable': results_b.params.index,
    'Coefficient': results_b.params.values,
    'Std_Error': results_b.std_errors.values if hasattr(results_b, 'std_errors') else results_b.bse.values,
    'P_Value': results_b.pvalues.values,
    'CI_Lower': results_b.conf_int()[0].values,
    'CI_Upper': results_b.conf_int()[1].values
})
results_b_df.to_csv('outputs/tables/reg_model_b_outcomes.csv', index=False)
print("\n✓ Saved: outputs/tables/reg_model_b_outcomes.csv")

In [ ]:
# MODEL C: Desert Indicator Model
print("\n" + "=" * 60)
print("MODEL C: Desert Indicator Model")
print("=" * 60)

X3_vars = ['desert_q_def2'] + controls

if HAS_PANELOLS:
    Y3 = reg_data_indexed['pqi_mean_rate']
    X3 = reg_data_indexed[X3_vars]
    model_c = PanelOLS(Y3, X3, entity_effects=True, time_effects=True)
    results_c = model_c.fit(cov_type='clustered', cluster_entity=True)
else:
    # Use simple OLS with controls
    Y3 = reg_data['pqi_mean_rate']
    X3 = reg_data[X3_vars].copy()
    X3 = sm.add_constant(X3)
    model_c = OLS(Y3, X3)
    results_c = model_c.fit()

print(results_c.summary())

# Save results
results_c_df = pd.DataFrame({
    'Variable': results_c.params.index,
    'Coefficient': results_c.params.values,
    'Std_Error': results_c.std_errors.values if hasattr(results_c, 'std_errors') else results_c.bse.values,
    'P_Value': results_c.pvalues.values,
    'CI_Lower': results_c.conf_int()[0].values,
    'CI_Upper': results_c.conf_int()[1].values
})
results_c_df.to_csv('outputs/tables/reg_model_c_desert.csv', index=False)
print("\n✓ Saved: outputs/tables/reg_model_c_desert.csv")

In [ ]:
# Create regression summary markdown
# Extract values first
medi_cal_coef_a = results_a.params.get('medi_cal_share', np.nan)
shortage_coef_a = results_a.params.get('shortage_flag', np.nan)
pcp_coef_b = results_b.params.get('pcp_per_100k', np.nan)
medi_cal_coef_b = results_b.params.get('medi_cal_share', np.nan)
shortage_coef_b = results_b.params.get('shortage_flag', np.nan)
desert_coef_c = results_c.params.get('desert_q_def2', np.nan)
n_obs = len(reg_data_indexed) if HAS_PANELOLS else len(reg_data)
n_counties = reg_data['fips5'].nunique()

summary_md = f"""# Regression Results Summary

## Model A: Provider Supply Model
**Dependent Variable:** pcp_per_100k (Primary care physicians per 100,000)

**Key Findings:**
- Medi-Cal Share coefficient: {medi_cal_coef_a:.4f}
- Shortage Flag coefficient: {shortage_coef_a:.4f}
- N observations: {n_obs}
- N counties: {n_counties}

## Model B: Outcomes Model
**Dependent Variable:** pqi_mean_rate (Mean PQI rate)

**Key Findings:**
- PCP per 100k coefficient: {pcp_coef_b:.4f}
- Medi-Cal Share coefficient: {medi_cal_coef_b:.4f}
- Shortage Flag coefficient: {shortage_coef_b:.4f}
- N observations: {n_obs}
- N counties: {n_counties}

## Model C: Desert Indicator Model
**Dependent Variable:** pqi_mean_rate (Mean PQI rate)

**Key Findings:**
- Desert (Quartile Def2) coefficient: {desert_coef_c:.4f}
- N observations: {n_obs}
- N counties: {n_counties}

## Interpretation

### Model A Interpretation:
A +0.10 increase in medi_cal_share is associated with Δ {medi_cal_coef_a * 0.10:.2f} pcp_per_100k.

### Model B Interpretation:
An increase of +10 pcp_per_100k is associated with Δ {pcp_coef_b * 10:.2f} pqi_mean_rate.

**Note:** Standard errors are clustered at the county level.
"""

with open('outputs/tables/regression_summary.md', 'w') as f:
    f.write(summary_md)

print("✓ Saved: outputs/tables/regression_summary.md")

## 12) Robustness / Sensitivity

In [ ]:
# Sensitivity checks
sensitivity_results = []

# 1. Alternative desert definitions
print("Running sensitivity checks...")

# Model B with alternative definitions
for desert_var in ['desert_q_def1', 'desert_thr_def1', 'desert_thr_def2']:
    if desert_var in reg_data.columns and reg_data[desert_var].sum() > 0:
        Y_sens = reg_data['pqi_mean_rate']
        X_sens_vars = [desert_var] + controls
        X_sens = reg_data[X_sens_vars].copy()
        
        if HAS_PANELOLS:
            Y_sens_idx = reg_data_indexed['pqi_mean_rate']
            X_sens_idx = reg_data_indexed[X_sens_vars]
            model_sens = PanelOLS(Y_sens_idx, X_sens_idx, entity_effects=True, time_effects=True)
            results_sens = model_sens.fit(cov_type='clustered', cluster_entity=True)
        else:
            # Simple OLS without fixed effects
            X_sens = sm.add_constant(X_sens)
            model_sens = OLS(Y_sens, X_sens)
            results_sens = model_sens.fit()
        
        coef = results_sens.params.get(desert_var, np.nan)
        se = results_sens.std_errors.get(desert_var, np.nan) if hasattr(results_sens, 'std_errors') else results_sens.bse.get(desert_var, np.nan)
        pval = results_sens.pvalues.get(desert_var, np.nan)
        
        sensitivity_results.append({
            'Specification': f'Model B - {desert_var}',
            'Coefficient': coef,
            'Std_Error': se,
            'P_Value': pval
        })
        print(f"  {desert_var}: coef={coef:.4f}, se={se:.4f}, p={pval:.4f}")
    else:
        print(f"  Skipping {desert_var} - no variation in data")

# 2. Winsorize outliers
print("\n  Running winsorized model...")
reg_data_winsor = reg_data.copy()
reg_data_winsor['medi_cal_share_w'] = reg_data_winsor['medi_cal_share'].clip(
    lower=reg_data_winsor['medi_cal_share'].quantile(0.01),
    upper=reg_data_winsor['medi_cal_share'].quantile(0.99)
)
reg_data_winsor['pqi_mean_rate_w'] = reg_data_winsor['pqi_mean_rate'].clip(
    lower=reg_data_winsor['pqi_mean_rate'].quantile(0.01),
    upper=reg_data_winsor['pqi_mean_rate'].quantile(0.99)
)

Y_w = reg_data_winsor['pqi_mean_rate_w']
X_w_vars = ['pcp_per_100k', 'medi_cal_share_w', 'shortage_flag'] + controls
X_w = reg_data_winsor[X_w_vars].copy()

if HAS_PANELOLS:
    reg_data_winsor_indexed = reg_data_winsor.set_index(['fips5', 'year']).sort_index()
    Y_w = reg_data_winsor_indexed['pqi_mean_rate_w']
    X_w = reg_data_winsor_indexed[X_w_vars]
    model_w = PanelOLS(Y_w, X_w, entity_effects=True, time_effects=True)
    results_w = model_w.fit(cov_type='clustered', cluster_entity=True)
else:
    # Simple OLS without fixed effects
    X_w = sm.add_constant(X_w)
    model_w = OLS(Y_w, X_w)
    results_w = model_w.fit()

sensitivity_results.append({
    'Specification': 'Model B - Winsorized (1st/99th percentile)',
    'Coefficient': results_w.params.get('pcp_per_100k', np.nan),
    'Std_Error': results_w.std_errors.get('pcp_per_100k', np.nan) if hasattr(results_w, 'std_errors') else results_w.bse.get('pcp_per_100k', np.nan),
    'P_Value': results_w.pvalues.get('pcp_per_100k', np.nan)
})
print(f"  Winsorized: coef={results_w.params.get('pcp_per_100k', np.nan):.4f}")

# Save sensitivity results
sensitivity_df = pd.DataFrame(sensitivity_results)
sensitivity_df.to_csv('outputs/tables/sensitivity_checks.csv', index=False)
print("\n✓ Saved: outputs/tables/sensitivity_checks.csv")
print("\nSensitivity Results:")
print(sensitivity_df.to_string())

## 13) Appendix Diagnostics

In [ ]:
# Missingness table
missingness_table = summarize_missingness(master_panel, "Master Panel")
missingness_table.to_csv('outputs/tables/missingness_table.csv')
print("\n✓ Saved: outputs/tables/missingness_table.csv")

In [ ]:
# Correlation matrix of main variables
main_vars = ['medi_cal_share', 'pcp_per_100k', 'pqi_mean_rate', 'shortage_flag',
            'poverty_pct', 'unemp_pct', 'age65_pct', 'hispanic_pct', 'bachelors_pct']
corr_data = master_panel[main_vars].dropna()
corr_matrix = corr_data.corr()

corr_matrix.to_csv('outputs/tables/correlation_matrix.csv')
print("✓ Saved: outputs/tables/correlation_matrix.csv")
print("\nCorrelation Matrix (main variables):")
print(corr_matrix.round(3))

In [ ]:
# County coverage table by year
coverage_by_year = master_panel.groupby('year').agg({
    'fips5': 'nunique',
    'medi_cal_share': 'count',
    'pcp_per_100k': 'count',
    'pqi_mean_rate': 'count'
}).reset_index()
coverage_by_year.columns = ['Year', 'N_Counties', 'N_MediCal', 'N_PCP', 'N_PQI']

coverage_by_year.to_csv('outputs/tables/county_coverage_by_year.csv', index=False)
print("✓ Saved: outputs/tables/county_coverage_by_year.csv")
print("\nCounty Coverage by Year:")
print(coverage_by_year.to_string())

In [ ]:
# Create results summary
# Extract values first to avoid f-string syntax issues
medi_cal_share_coef_a = results_a.params.get('medi_cal_share', np.nan)
medi_cal_share_se_a = results_a.std_errors.get('medi_cal_share', np.nan) if hasattr(results_a, 'std_errors') else results_a.bse.get('medi_cal_share', np.nan)
shortage_flag_coef_a = results_a.params.get('shortage_flag', np.nan)
shortage_flag_se_a = results_a.std_errors.get('shortage_flag', np.nan) if hasattr(results_a, 'std_errors') else results_a.bse.get('shortage_flag', np.nan)

pcp_coef_b = results_b.params.get('pcp_per_100k', np.nan)
pcp_se_b = results_b.std_errors.get('pcp_per_100k', np.nan) if hasattr(results_b, 'std_errors') else results_b.bse.get('pcp_per_100k', np.nan)
medi_cal_share_coef_b = results_b.params.get('medi_cal_share', np.nan)
medi_cal_share_se_b = results_b.std_errors.get('medi_cal_share', np.nan) if hasattr(results_b, 'std_errors') else results_b.bse.get('medi_cal_share', np.nan)
shortage_flag_coef_b = results_b.params.get('shortage_flag', np.nan)
shortage_flag_se_b = results_b.std_errors.get('shortage_flag', np.nan) if hasattr(results_b, 'std_errors') else results_b.bse.get('shortage_flag', np.nan)

desert_coef_c = results_c.params.get('desert_q_def2', np.nan)
desert_se_c = results_c.std_errors.get('desert_q_def2', np.nan) if hasattr(results_c, 'std_errors') else results_c.bse.get('desert_q_def2', np.nan)

physician_supply_text = 'Physician supply is time-invariant' if not IS_TIME_VARYING else 'Physician supply varies by year'
physician_supply_note = 'This limits the ability to examine within-county changes over time.' if not IS_TIME_VARYING else ''
shortage_text = 'Shortage designation is time-invariant' if not IS_SHORTAGE_TIME_VARYING else 'Shortage designation varies by year'

results_summary = f"""# California Medi-Cal Deserts Capstone Project - Results Summary

## Data Sources
- County crosswalk: county name.xlsx
- Medi-Cal enrollment: medi-cal-enrollment-dashboard-data.csv
- Population: E4 estiamtes.xlsx
- Physician supply: physicians-actively-working-by-specialty-and-patient-care-hours.xlsx
- Shortage designation: Primary CAre Shortage .csv
- PQI outcomes: PQI.csv
- ACS controls: demoACS.csv, educACS.csv, EconACS.csv

## Final Dataset
- **Year range:** {master_panel['year'].min():.0f} - {master_panel['year'].max():.0f}
- **N counties:** {master_panel['fips5'].nunique()}
- **N county-years:** {len(master_panel)}
- **Desert definition (main results):** Quartile-based Definition 2 (high Medi-Cal & (low PCP OR shortage))

## Key Descriptive Findings

### Medi-Cal Intensity
- Mean Medi-Cal share: {master_panel['medi_cal_share'].mean():.3f}
- Median Medi-Cal share: {master_panel['medi_cal_share'].median():.3f}
- Range: {master_panel['medi_cal_share'].min():.3f} - {master_panel['medi_cal_share'].max():.3f}

### Primary Care Supply
- Mean PCP per 100k: {master_panel['pcp_per_100k'].mean():.2f}
- Median PCP per 100k: {master_panel['pcp_per_100k'].median():.2f}
- Range: {master_panel['pcp_per_100k'].min():.2f} - {master_panel['pcp_per_100k'].max():.2f}
- **Time-varying:** {'Yes' if IS_TIME_VARYING else 'No (time-invariant)'}

### Preventable Hospitalizations (PQI)
- Mean PQI rate: {master_panel['pqi_mean_rate'].mean():.2f}
- Median PQI rate: {master_panel['pqi_mean_rate'].median():.2f}
- Range: {master_panel['pqi_mean_rate'].min():.2f} - {master_panel['pqi_mean_rate'].max():.2f}

### Shortage Designations
- Counties with shortage designation: {(master_panel['shortage_flag'] == 1).sum()} county-years
- **Time-varying:** {'Yes' if IS_SHORTAGE_TIME_VARYING else 'No (time-invariant)'}

### Desert Classifications
- Desert (Quartile Def1): {master_panel['desert_q_def1'].sum()} county-years
- Desert (Quartile Def2): {master_panel['desert_q_def2'].sum()} county-years
- Desert (Threshold Def1): {master_panel['desert_thr_def1'].sum()} county-years
- Desert (Threshold Def2): {master_panel['desert_thr_def2'].sum()} county-years

## Key Regression Findings (with Clustered Standard Errors)

### Model A: Provider Supply Model
**Dependent Variable:** pcp_per_100k

- **Medi-Cal Share:** {medi_cal_share_coef_a:.4f} (SE: {medi_cal_share_se_a:.4f})
- **Shortage Flag:** {shortage_flag_coef_a:.4f} (SE: {shortage_flag_se_a:.4f})

**Interpretation:** A +0.10 increase in medi_cal_share is associated with Δ {medi_cal_share_coef_a * 0.10:.2f} pcp_per_100k.

### Model B: Outcomes Model
**Dependent Variable:** pqi_mean_rate

- **PCP per 100k:** {pcp_coef_b:.4f} (SE: {pcp_se_b:.4f})
- **Medi-Cal Share:** {medi_cal_share_coef_b:.4f} (SE: {medi_cal_share_se_b:.4f})
- **Shortage Flag:** {shortage_flag_coef_b:.4f} (SE: {shortage_flag_se_b:.4f})

**Interpretation:** An increase of +10 pcp_per_100k is associated with Δ {pcp_coef_b * 10:.2f} pqi_mean_rate.

### Model C: Desert Indicator Model
**Dependent Variable:** pqi_mean_rate

- **Desert (Quartile Def2):** {desert_coef_c:.4f} (SE: {desert_se_c:.4f})

## Limitations

1. **ACS Controls:** ACS controls are likely single-year (baseline) and treated as time-invariant. This may not capture time-varying demographic/economic changes.

2. **Physician Supply:** {physician_supply_text}. {physician_supply_note}

3. **Shortage Designation:** {shortage_text}.

4. **Ecological Inference:** Results are at the county-year level. Individual-level relationships may differ.

5. **Causality:** These are observational associations. Causal interpretation requires additional assumptions and methods.

## Output Files Generated

### Data Files (outputs/data/)
- county_crosswalk_clean.csv
- pop_e4_clean.csv
- medical_enrollment_clean.csv
- physician_supply_clean.csv
- shortage_clean.csv
- acs_controls_clean.csv
- pqi_long_clean.csv
- pqi_county_year_clean.csv
- ca_master_panel.csv
- ca_variable_dictionary.csv

### Figures (outputs/figures/)
- hist_medi_cal_share.png
- hist_pcp_per_100k.png
- time_series_statewide.png
- scatter_medi_cal_vs_pcp.png
- scatter_pcp_vs_pqi.png
- bar_top_bottom_pqi.png
- maps_california.png (if shapefile available)

### Tables (outputs/tables/)
- primary_care_specialty_definition.csv
- reg_model_a_supply.csv
- reg_model_b_outcomes.csv
- reg_model_c_desert.csv
- regression_summary.md
- sensitivity_checks.csv
- missingness_table.csv
- correlation_matrix.csv
- county_coverage_by_year.csv
"""

with open('outputs/results_summary.md', 'w') as f:
    f.write(results_summary)

print("✓ Saved: outputs/results_summary.md")
print("\n" + "=" * 60)
print("ANALYSIS COMPLETE")
print("=" * 60)
print("\nAll outputs saved to outputs/ directory.")
print("See outputs/results_summary.md for full summary.")

In [ ]:
# Create README
readme_content = """# California Medi-Cal Deserts Capstone Project

## How to Run

1. Ensure all required data files are in the same directory as this notebook:
   - county name.xlsx
   - medi-cal-enrollment-dashboard-data.csv
   - E4 estiamtes.xlsx
   - physicians-actively-working-by-specialty-and-patient-care-hours.xlsx
   - Primary CAre Shortage .csv
   - PQI.csv
   - demoACS.csv
   - educACS.csv
   - EconACS.csv

2. Optional files (will be loaded if present):
   - pqi-physicians-specialities-list.xlsx
   - Other physician activity files
   - County shapefile (for maps)

3. Run all cells in order (Cell > Run All)

4. Outputs will be saved to:
   - outputs/data/ - Cleaned datasets
   - outputs/figures/ - All figures
   - outputs/tables/ - Regression results and diagnostic tables

## Dependencies

Required:
- pandas
- numpy
- matplotlib
- openpyxl (for Excel files)

Optional but recommended:
- linearmodels (for PanelOLS)
- statsmodels (fallback if linearmodels not available)
- geopandas (for maps, if shapefile available)

## Project Structure

The notebook follows this structure:
0. Setup & Reproducibility
1. Load Raw Data
2. Build Clean County Crosswalk
3. Clean Population
4. Clean Medi-Cal Enrollment
5. Clean Physician Supply
6. Clean Shortage File
7. Clean ACS Controls
8. Clean PQI and Build Outcomes
9. Build Master Panel
10. Descriptive Figures
11. Regression Analyses
12. Robustness / Sensitivity
13. Appendix Diagnostics
14. Final Outputs

## Key Outputs

- **ca_master_panel.csv**: Final county-year panel dataset
- **results_summary.md**: Comprehensive results summary
- **regression_summary.md**: Regression results interpretation
- All figures and tables in outputs/ subdirectories

## Notes

- The notebook automatically detects whether physician supply and shortage designations are time-varying
- Models adapt accordingly (panel vs cross-sectional)
- Standard errors are clustered at the county level
- Desert definitions use both quartile-based and threshold-based approaches
"""

with open('outputs/README.md', 'w') as f:
    f.write(readme_content)

print("✓ Saved: outputs/README.md")
print("\n" + "=" * 60)
print("NOTEBOOK COMPLETE - ALL SECTIONS IMPLEMENTED")
print("=" * 60)

## 15) ED Encounters Analysis (New Data Integration)

This section integrates Emergency Department encounter data (2012-2023) to:
1. Analyze ED utilization patterns across counties
2. Test if high ED use indicates lack of primary care access
3. Examine ED as a potential mediator between Medi-Cal share and PQI
4. Extend time series analysis beyond the Medi-Cal enrollment constraint

In [ ]:
# Load ED Encounters Data
print("=" * 60)
print("LOADING ED ENCOUNTERS DATA")
print("=" * 60)

encounters_file = 'encounters.csv'
if os.path.exists(encounters_file):
    df_encounters = pd.read_csv(encounters_file)
    print(f"✓ Loaded encounters.csv: {df_encounters.shape}")
    print(f"\nColumns: {list(df_encounters.columns)}")
    print(f"\nYear range: {df_encounters['year'].min()} - {df_encounters['year'].max()}")
    print(f"Unique hospitals: {df_encounters['oshpd_id'].nunique()}")
    print(f"Unique counties: {df_encounters['county_name'].nunique()}")
    print(f"\nEncounter types: {df_encounters['type'].unique()}")
    print(f"\nER Service Levels: {df_encounters['er_service_level_desc'].unique()}")
    print(f"\nSample data:")
    display(df_encounters.head(10))
else:
    print("ERROR: encounters.csv not found!")
    df_encounters = None

In [ ]:
# Clean and Aggregate ED Encounters to County-Year Level
print("=" * 60)
print("AGGREGATING ED ENCOUNTERS TO COUNTY-YEAR LEVEL")
print("=" * 60)

if df_encounters is not None:
    # Standardize county names
    df_encounters['county_clean'] = df_encounters['county_name'].apply(standardize_county_name)
    
    # Pivot to get ED_Visit and ED_Admit as separate columns
    encounters_pivot = df_encounters.pivot_table(
        index=['year', 'county_clean'],
        columns='type',
        values='count',
        aggfunc='sum'
    ).reset_index()
    
    # Flatten column names
    encounters_pivot.columns.name = None
    encounters_pivot = encounters_pivot.rename(columns={
        'ED_Admit': 'ed_admissions',
        'ED_Visit': 'ed_visits'
    })
    
    # Fill NaN with 0 for missing categories
    encounters_pivot['ed_admissions'] = encounters_pivot['ed_admissions'].fillna(0)
    encounters_pivot['ed_visits'] = encounters_pivot['ed_visits'].fillna(0)
    
    # Calculate derived metrics
    encounters_pivot['ed_total'] = encounters_pivot['ed_visits'] + encounters_pivot['ed_admissions']
    encounters_pivot['ed_admit_rate'] = encounters_pivot['ed_admissions'] / encounters_pivot['ed_visits']
    encounters_pivot['ed_admit_rate'] = encounters_pivot['ed_admit_rate'].replace([np.inf, -np.inf], np.nan)
    
    # Merge with crosswalk to get fips5
    encounters_county_year = encounters_pivot.merge(
        crosswalk_clean[['county_name_clean', 'fips5']],
        left_on='county_clean',
        right_on='county_name_clean',
        how='left'
    )
    
    # Check for unmatched counties
    unmatched = encounters_county_year[encounters_county_year['fips5'].isna()]['county_clean'].unique()
    if len(unmatched) > 0:
        print(f"⚠️ Warning: {len(unmatched)} counties could not be matched to FIPS:")
        print(unmatched[:10])
    
    # Drop unmatched and clean up
    encounters_county_year = encounters_county_year[encounters_county_year['fips5'].notna()].copy()
    encounters_county_year = encounters_county_year[['fips5', 'year', 'ed_visits', 'ed_admissions', 
                                                       'ed_total', 'ed_admit_rate']]
    
    print(f"\n✓ Created county-year ED encounters dataset:")
    print(f"  Shape: {encounters_county_year.shape}")
    print(f"  Year range: {encounters_county_year['year'].min()} - {encounters_county_year['year'].max()}")
    print(f"  Unique counties: {encounters_county_year['fips5'].nunique()}")
    
    # Summary stats
    print(f"\n--- ED Summary Statistics ---")
    print(encounters_county_year[['ed_visits', 'ed_admissions', 'ed_total', 'ed_admit_rate']].describe())
    
    # Save
    encounters_county_year.to_csv('outputs/data/ed_encounters_county_year.csv', index=False)
    print(f"\n✓ Saved: outputs/data/ed_encounters_county_year.csv")
else:
    encounters_county_year = None

In [ ]:
# Calculate ED Per Capita Rates and Merge with Population
print("=" * 60)
print("CALCULATING ED PER CAPITA RATES")
print("=" * 60)

if encounters_county_year is not None:
    # Merge with population data
    ed_with_pop = encounters_county_year.merge(
        pop_year[['fips5', 'year', 'population']],
        on=['fips5', 'year'],
        how='left'
    )
    
    # Calculate per capita rates (per 1,000 population)
    ed_with_pop['ed_visits_per_1k'] = (ed_with_pop['ed_visits'] / ed_with_pop['population']) * 1000
    ed_with_pop['ed_admits_per_1k'] = (ed_with_pop['ed_admissions'] / ed_with_pop['population']) * 1000
    ed_with_pop['ed_total_per_1k'] = (ed_with_pop['ed_total'] / ed_with_pop['population']) * 1000
    
    # Check population merge success
    pop_matched = ed_with_pop['population'].notna().sum()
    print(f"✓ Population matched for {pop_matched}/{len(ed_with_pop)} records ({100*pop_matched/len(ed_with_pop):.1f}%)")
    
    # Summary of per-capita rates
    print(f"\n--- ED Per 1,000 Population Statistics ---")
    print(ed_with_pop[['ed_visits_per_1k', 'ed_admits_per_1k', 'ed_total_per_1k', 'ed_admit_rate']].describe())
    
    # Save enhanced dataset
    ed_with_pop.to_csv('outputs/data/ed_encounters_with_rates.csv', index=False)
    print(f"\n✓ Saved: outputs/data/ed_encounters_with_rates.csv")
    
    encounters_county_year = ed_with_pop.copy()
else:
    print("No encounters data to process.")

In [ ]:
# ED Utilization Time Series (2012-2023)
print("=" * 60)
print("ED UTILIZATION TIME SERIES ANALYSIS")
print("=" * 60)

if encounters_county_year is not None:
    # Aggregate to statewide by year (weighted by population)
    ed_state_year = encounters_county_year.groupby('year').agg({
        'ed_visits': 'sum',
        'ed_admissions': 'sum',
        'ed_total': 'sum',
        'population': 'sum'
    }).reset_index()
    
    # Calculate statewide rates
    ed_state_year['ed_visits_per_1k'] = (ed_state_year['ed_visits'] / ed_state_year['population']) * 1000
    ed_state_year['ed_admits_per_1k'] = (ed_state_year['ed_admissions'] / ed_state_year['population']) * 1000
    ed_state_year['ed_admit_rate'] = ed_state_year['ed_admissions'] / ed_state_year['ed_visits']
    
    print("Statewide ED Trends (2012-2023):")
    print(ed_state_year[['year', 'ed_visits', 'ed_visits_per_1k', 'ed_admit_rate']].to_string(index=False))
    
    # Create time series figure
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Total ED Visits
    ax1 = axes[0, 0]
    ax1.plot(ed_state_year['year'], ed_state_year['ed_visits']/1e6, 'b-o', linewidth=2, markersize=6)
    ax1.set_xlabel('Year', fontsize=11)
    ax1.set_ylabel('ED Visits (Millions)', fontsize=11)
    ax1.set_title('Total Statewide ED Visits', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.axvline(x=2020, color='red', linestyle='--', alpha=0.5, label='COVID-19')
    ax1.legend()
    
    # Plot 2: ED Visits per 1,000
    ax2 = axes[0, 1]
    ax2.plot(ed_state_year['year'], ed_state_year['ed_visits_per_1k'], 'g-o', linewidth=2, markersize=6)
    ax2.set_xlabel('Year', fontsize=11)
    ax2.set_ylabel('ED Visits per 1,000 Population', fontsize=11)
    ax2.set_title('ED Utilization Rate (per 1,000)', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.axvline(x=2020, color='red', linestyle='--', alpha=0.5, label='COVID-19')
    ax2.legend()
    
    # Plot 3: ED Admission Rate
    ax3 = axes[1, 0]
    ax3.plot(ed_state_year['year'], ed_state_year['ed_admit_rate']*100, 'r-o', linewidth=2, markersize=6)
    ax3.set_xlabel('Year', fontsize=11)
    ax3.set_ylabel('ED Admission Rate (%)', fontsize=11)
    ax3.set_title('ED-to-Admission Conversion Rate', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.axvline(x=2020, color='red', linestyle='--', alpha=0.5, label='COVID-19')
    ax3.legend()
    
    # Plot 4: ED Admissions per 1,000
    ax4 = axes[1, 1]
    ax4.plot(ed_state_year['year'], ed_state_year['ed_admits_per_1k'], 'm-o', linewidth=2, markersize=6)
    ax4.set_xlabel('Year', fontsize=11)
    ax4.set_ylabel('ED Admissions per 1,000 Population', fontsize=11)
    ax4.set_title('ED Admissions Rate (per 1,000)', fontsize=12, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    ax4.axvline(x=2020, color='red', linestyle='--', alpha=0.5, label='COVID-19')
    ax4.legend()
    
    plt.tight_layout()
    plt.savefig('outputs/figures/ed_trends_2012_2023.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: outputs/figures/ed_trends_2012_2023.png")
    
    # Save statewide data
    ed_state_year.to_csv('outputs/data/ed_statewide_trends.csv', index=False)
    print("✓ Saved: outputs/data/ed_statewide_trends.csv")
else:
    print("No ED data available for time series analysis.")

In [ ]:
# County-Level ED Variation Analysis
print("=" * 60)
print("COUNTY-LEVEL ED VARIATION ANALYSIS")
print("=" * 60)

if encounters_county_year is not None:
    # Get latest year data (2023) for comparison
    latest_year = encounters_county_year['year'].max()
    ed_latest = encounters_county_year[encounters_county_year['year'] == latest_year].copy()
    
    # Merge with crosswalk to get county names
    ed_latest = ed_latest.merge(
        crosswalk_clean[['fips5', 'county_name_clean']],
        on='fips5',
        how='left'
    )
    
    # Sort by ED visit rate
    ed_latest_sorted = ed_latest.sort_values('ed_visits_per_1k', ascending=False)
    
    print(f"\n--- Top 10 Counties by ED Visits per 1,000 ({latest_year}) ---")
    print(ed_latest_sorted[['county_name_clean', 'ed_visits_per_1k', 'ed_admit_rate', 'population']].head(10).to_string(index=False))
    
    print(f"\n--- Bottom 10 Counties by ED Visits per 1,000 ({latest_year}) ---")
    print(ed_latest_sorted[['county_name_clean', 'ed_visits_per_1k', 'ed_admit_rate', 'population']].tail(10).to_string(index=False))
    
    # Create county comparison figure
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Plot 1: Top/Bottom 15 counties by ED visit rate
    ax1 = axes[0]
    top_15 = ed_latest_sorted.head(15)
    bottom_15 = ed_latest_sorted.tail(15)
    combined = pd.concat([top_15, bottom_15])
    
    colors = ['#d62728' if x in top_15['fips5'].values else '#2ca02c' for x in combined['fips5']]
    bars = ax1.barh(range(len(combined)), combined['ed_visits_per_1k'], color=colors)
    ax1.set_yticks(range(len(combined)))
    ax1.set_yticklabels(combined['county_name_clean'], fontsize=8)
    ax1.set_xlabel('ED Visits per 1,000 Population', fontsize=11)
    ax1.set_title(f'Top 15 (Red) and Bottom 15 (Green) Counties\nby ED Utilization Rate ({latest_year})', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='x')
    
    # Plot 2: ED Admission Rate vs ED Visit Rate scatter
    ax2 = axes[1]
    scatter = ax2.scatter(ed_latest['ed_visits_per_1k'], ed_latest['ed_admit_rate']*100, 
                          s=ed_latest['population']/10000, alpha=0.6, c='steelblue', edgecolors='black', linewidth=0.5)
    ax2.set_xlabel('ED Visits per 1,000 Population', fontsize=11)
    ax2.set_ylabel('ED Admission Rate (%)', fontsize=11)
    ax2.set_title(f'ED Visit Rate vs Admission Rate by County ({latest_year})\n(Bubble size = Population)', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Label extreme outliers
    for idx, row in ed_latest.nlargest(5, 'ed_visits_per_1k').iterrows():
        ax2.annotate(row['county_name_clean'], (row['ed_visits_per_1k'], row['ed_admit_rate']*100),
                     fontsize=8, alpha=0.8, ha='left')
    
    plt.tight_layout()
    plt.savefig('outputs/figures/ed_county_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: outputs/figures/ed_county_comparison.png")
else:
    print("No ED data available for county comparison.")

In [ ]:
# Merge ED Data with Master Panel for Regression Analysis
print("=" * 60)
print("MERGING ED DATA WITH MASTER PANEL")
print("=" * 60)

if encounters_county_year is not None and master_panel is not None:
    # Get year from master panel
    panel_year = master_panel['year'].iloc[0] if 'year' in master_panel.columns else 2020
    print(f"Master panel year: {panel_year}")
    
    # Filter ED data to matching year
    ed_for_merge = encounters_county_year[encounters_county_year['year'] == panel_year].copy()
    print(f"ED records for {panel_year}: {len(ed_for_merge)}")
    
    # Merge with master panel
    master_with_ed = master_panel.merge(
        ed_for_merge[['fips5', 'ed_visits', 'ed_admissions', 'ed_visits_per_1k', 'ed_admits_per_1k', 'ed_admit_rate']],
        on='fips5',
        how='left'
    )
    
    # Check merge success
    ed_matched = master_with_ed['ed_visits_per_1k'].notna().sum()
    print(f"✓ ED data matched for {ed_matched}/{len(master_with_ed)} counties ({100*ed_matched/len(master_with_ed):.1f}%)")
    
    # Summary of ED metrics in merged data
    print(f"\n--- ED Metrics in Master Panel ({panel_year}) ---")
    print(master_with_ed[['ed_visits_per_1k', 'ed_admits_per_1k', 'ed_admit_rate']].describe())
    
    # Correlation with key variables
    print(f"\n--- Correlations with ED Visits per 1k ---")
    ed_corr_vars = ['pqi_mean_rate', 'medi_cal_share', 'pcp_per_100k', 'poverty_pct']
    available_vars = [v for v in ed_corr_vars if v in master_with_ed.columns]
    for var in available_vars:
        corr = master_with_ed[['ed_visits_per_1k', var]].corr().iloc[0, 1]
        print(f"  ED visits per 1k vs {var}: {corr:.3f}")
    
    # Save enhanced panel
    master_with_ed.to_csv('outputs/data/ca_master_panel_with_ed.csv', index=False)
    print(f"\n✓ Saved: outputs/data/ca_master_panel_with_ed.csv")
else:
    master_with_ed = None
    print("Cannot merge - missing ED data or master panel.")

In [ ]:
# ED Regression Analysis
print("=" * 60)
print("ED REGRESSION ANALYSIS")
print("=" * 60)

import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

if master_with_ed is not None:
    # Prepare data - drop NaN for analysis
    ed_analysis = master_with_ed.dropna(subset=['ed_visits_per_1k', 'pqi_mean_rate', 'medi_cal_share']).copy()
    print(f"Analysis sample: {len(ed_analysis)} counties")
    
    # Define controls
    control_vars = ['poverty_pct', 'unemp_pct', 'age65_pct', 'hispanic_pct', 'bachelors_pct']
    available_controls = [v for v in control_vars if v in ed_analysis.columns and ed_analysis[v].notna().sum() > 0]
    print(f"Available controls: {available_controls}")
    
    results_list = []
    
    # =========================================
    # Model E1: ED Visits as Outcome
    # Does Medi-Cal share predict ED utilization?
    # =========================================
    print("\n" + "=" * 50)
    print("MODEL E1: What predicts ED Utilization?")
    print("DV: ED Visits per 1,000 Population")
    print("=" * 50)
    
    Y_e1 = ed_analysis['ed_visits_per_1k']
    X_vars_e1 = ['medi_cal_share']
    if 'pcp_per_100k' in ed_analysis.columns:
        X_vars_e1.append('pcp_per_100k')
    X_vars_e1.extend(available_controls)
    
    X_e1 = ed_analysis[X_vars_e1].dropna()
    Y_e1 = Y_e1.loc[X_e1.index]
    X_e1 = sm.add_constant(X_e1)
    
    model_e1 = OLS(Y_e1, X_e1).fit(cov_type='HC1')
    print(model_e1.summary2().tables[1])
    
    # Store results
    results_list.append({
        'Model': 'E1: ED Utilization',
        'DV': 'ED Visits per 1k',
        'Medi-Cal Coef': model_e1.params.get('medi_cal_share', np.nan),
        'Medi-Cal P-value': model_e1.pvalues.get('medi_cal_share', np.nan),
        'R2': model_e1.rsquared,
        'N': int(model_e1.nobs)
    })
    
    # Interpretation
    mc_coef_e1 = model_e1.params.get('medi_cal_share', 0)
    print(f"\n**Interpretation:** A +10 percentage point increase in Medi-Cal share")
    print(f"is associated with {mc_coef_e1 * 0.10:.1f} more ED visits per 1,000 population.")
    
    # =========================================
    # Model E2: ED as Mediator between Medi-Cal and PQI
    # Does ED utilization explain the Medi-Cal → PQI relationship?
    # =========================================
    print("\n" + "=" * 50)
    print("MODEL E2: PQI with ED as Additional Predictor")
    print("DV: PQI Mean Rate (Preventable Hospitalizations)")
    print("=" * 50)
    
    # First: PQI without ED (baseline)
    Y_e2 = ed_analysis['pqi_mean_rate']
    X_vars_baseline = ['medi_cal_share'] + available_controls
    if 'pcp_per_100k' in ed_analysis.columns:
        X_vars_baseline.insert(1, 'pcp_per_100k')
    
    X_baseline = ed_analysis[X_vars_baseline].dropna()
    Y_baseline = Y_e2.loc[X_baseline.index]
    X_baseline_const = sm.add_constant(X_baseline)
    
    model_baseline = OLS(Y_baseline, X_baseline_const).fit(cov_type='HC1')
    mc_coef_baseline = model_baseline.params.get('medi_cal_share', 0)
    
    # Second: PQI with ED
    X_vars_with_ed = ['medi_cal_share', 'ed_visits_per_1k'] + available_controls
    if 'pcp_per_100k' in ed_analysis.columns:
        X_vars_with_ed.insert(2, 'pcp_per_100k')
    
    X_with_ed = ed_analysis[X_vars_with_ed].dropna()
    Y_with_ed = Y_e2.loc[X_with_ed.index]
    X_with_ed_const = sm.add_constant(X_with_ed)
    
    model_with_ed = OLS(Y_with_ed, X_with_ed_const).fit(cov_type='HC1')
    mc_coef_with_ed = model_with_ed.params.get('medi_cal_share', 0)
    
    print("Without ED control:")
    print(f"  Medi-Cal coef: {mc_coef_baseline:.2f}")
    print("\nWith ED control:")
    print(model_with_ed.summary2().tables[1])
    
    # Calculate mediation
    pct_mediated = (mc_coef_baseline - mc_coef_with_ed) / mc_coef_baseline * 100 if mc_coef_baseline != 0 else np.nan
    print(f"\n**Mediation Analysis:**")
    print(f"  Medi-Cal coef without ED: {mc_coef_baseline:.2f}")
    print(f"  Medi-Cal coef with ED: {mc_coef_with_ed:.2f}")
    print(f"  % of effect mediated by ED: {pct_mediated:.1f}%")
    
    results_list.append({
        'Model': 'E2: PQI with ED',
        'DV': 'PQI Mean Rate',
        'Medi-Cal Coef': mc_coef_with_ed,
        'Medi-Cal P-value': model_with_ed.pvalues.get('medi_cal_share', np.nan),
        'R2': model_with_ed.rsquared,
        'N': int(model_with_ed.nobs)
    })
    
    # =========================================
    # Model E3: Does ED predict PQI independently?
    # =========================================
    print("\n" + "=" * 50)
    print("MODEL E3: ED → PQI Relationship")
    print("DV: PQI Mean Rate")
    print("=" * 50)
    
    ed_coef = model_with_ed.params.get('ed_visits_per_1k', 0)
    ed_pval = model_with_ed.pvalues.get('ed_visits_per_1k', np.nan)
    
    print(f"ED Visits per 1k coefficient: {ed_coef:.4f}")
    print(f"P-value: {ed_pval:.4f}")
    print(f"\n**Interpretation:** Each additional 10 ED visits per 1k population")
    print(f"is associated with {ed_coef * 10:.2f} higher PQI rate.")
    
    results_list.append({
        'Model': 'E3: ED → PQI',
        'DV': 'PQI Mean Rate',
        'Medi-Cal Coef': ed_coef,
        'Medi-Cal P-value': ed_pval,
        'R2': model_with_ed.rsquared,
        'N': int(model_with_ed.nobs)
    })
    
    # Save results
    ed_results_df = pd.DataFrame(results_list)
    ed_results_df.to_csv('outputs/tables/ed_regression_results.csv', index=False)
    print(f"\n✓ Saved: outputs/tables/ed_regression_results.csv")
    
else:
    print("Cannot run ED regressions - missing merged data.")

In [ ]:
# ED Utilization in Desert vs Non-Desert Counties
print("=" * 60)
print("ED UTILIZATION: DESERT vs NON-DESERT COMPARISON")
print("=" * 60)

if master_with_ed is not None:
    # Check if desert columns exist
    desert_cols = [c for c in master_with_ed.columns if 'desert' in c.lower()]
    print(f"Desert columns available: {desert_cols}")
    
    if 'desert_thr_def2' in master_with_ed.columns or 'desert_q_def2' in master_with_ed.columns:
        desert_col = 'desert_thr_def2' if 'desert_thr_def2' in master_with_ed.columns else 'desert_q_def2'
        
        # Filter to non-missing ED data
        ed_desert = master_with_ed[master_with_ed['ed_visits_per_1k'].notna()].copy()
        
        # Compare means
        desert_group = ed_desert.groupby(desert_col).agg({
            'ed_visits_per_1k': ['mean', 'std', 'count'],
            'ed_admits_per_1k': ['mean', 'std'],
            'ed_admit_rate': ['mean', 'std'],
            'pqi_mean_rate': ['mean', 'std']
        }).round(2)
        
        print(f"\n--- ED Metrics by Desert Status ({desert_col}) ---")
        print(desert_group)
        
        # Calculate difference
        non_desert = ed_desert[ed_desert[desert_col] == 0]
        desert = ed_desert[ed_desert[desert_col] == 1]
        
        print(f"\n--- Summary ---")
        print(f"Non-Desert counties: {len(non_desert)}")
        print(f"Desert counties: {len(desert)}")
        
        if len(desert) > 0 and len(non_desert) > 0:
            ed_diff = desert['ed_visits_per_1k'].mean() - non_desert['ed_visits_per_1k'].mean()
            pqi_diff = desert['pqi_mean_rate'].mean() - non_desert['pqi_mean_rate'].mean()
            
            print(f"\nED Visits per 1k: Desert = {desert['ed_visits_per_1k'].mean():.1f}, Non-Desert = {non_desert['ed_visits_per_1k'].mean():.1f}")
            print(f"  Difference: {ed_diff:+.1f} ({100*ed_diff/non_desert['ed_visits_per_1k'].mean():+.1f}%)")
            print(f"\nPQI Rate: Desert = {desert['pqi_mean_rate'].mean():.1f}, Non-Desert = {non_desert['pqi_mean_rate'].mean():.1f}")
            print(f"  Difference: {pqi_diff:+.1f} ({100*pqi_diff/non_desert['pqi_mean_rate'].mean():+.1f}%)")
        
        # Visualization
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Plot 1: ED Visits by Desert Status
        ax1 = axes[0]
        desert_means = [non_desert['ed_visits_per_1k'].mean(), desert['ed_visits_per_1k'].mean()]
        desert_stds = [non_desert['ed_visits_per_1k'].std(), desert['ed_visits_per_1k'].std()]
        bars1 = ax1.bar(['Non-Desert', 'Desert'], desert_means, yerr=desert_stds, capsize=5, 
                        color=['#2ca02c', '#d62728'], edgecolor='black', linewidth=1.5)
        ax1.set_ylabel('ED Visits per 1,000 Population', fontsize=11)
        ax1.set_title('ED Utilization by Desert Status', fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='y')
        
        # Plot 2: ED Admission Rate by Desert Status
        ax2 = axes[1]
        admit_means = [non_desert['ed_admit_rate'].mean()*100, desert['ed_admit_rate'].mean()*100]
        admit_stds = [non_desert['ed_admit_rate'].std()*100, desert['ed_admit_rate'].std()*100]
        bars2 = ax2.bar(['Non-Desert', 'Desert'], admit_means, yerr=admit_stds, capsize=5,
                        color=['#2ca02c', '#d62728'], edgecolor='black', linewidth=1.5)
        ax2.set_ylabel('ED Admission Rate (%)', fontsize=11)
        ax2.set_title('ED Admission Rate by Desert Status', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='y')
        
        # Plot 3: Scatter - ED vs PQI colored by desert
        ax3 = axes[2]
        ax3.scatter(non_desert['ed_visits_per_1k'], non_desert['pqi_mean_rate'], 
                   c='#2ca02c', alpha=0.6, s=80, label='Non-Desert', edgecolors='black', linewidth=0.5)
        ax3.scatter(desert['ed_visits_per_1k'], desert['pqi_mean_rate'], 
                   c='#d62728', alpha=0.6, s=80, label='Desert', edgecolors='black', linewidth=0.5)
        ax3.set_xlabel('ED Visits per 1,000 Population', fontsize=11)
        ax3.set_ylabel('PQI Mean Rate', fontsize=11)
        ax3.set_title('ED Utilization vs PQI by Desert Status', fontsize=12, fontweight='bold')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('outputs/figures/ed_desert_comparison.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("\n✓ Saved: outputs/figures/ed_desert_comparison.png")
    else:
        print("No desert classification columns found.")
else:
    print("No merged data available for desert comparison.")

In [ ]:
# Extended Time Series: ED and PQI (2012-2023)
print("=" * 60)
print("EXTENDED TIME SERIES: ED AND PQI (2012-2023)")
print("=" * 60)

if encounters_county_year is not None and 'pqi_county_year' in dir():
    # Merge ED with PQI for overlapping years
    ed_pqi_panel = encounters_county_year.merge(
        pqi_county_year[['fips5', 'year', 'pqi_mean_rate', 'pqi_sum_count', 'pqi_sum_population']],
        on=['fips5', 'year'],
        how='inner'
    )
    
    print(f"ED + PQI merged panel:")
    print(f"  Shape: {ed_pqi_panel.shape}")
    print(f"  Year range: {ed_pqi_panel['year'].min()} - {ed_pqi_panel['year'].max()}")
    print(f"  Unique counties: {ed_pqi_panel['fips5'].nunique()}")
    
    # Aggregate to statewide trends
    ed_pqi_state = ed_pqi_panel.groupby('year').agg({
        'ed_visits': 'sum',
        'ed_admissions': 'sum',
        'population': 'sum',
        'pqi_sum_count': 'sum',
        'pqi_sum_population': 'sum'
    }).reset_index()
    
    # Calculate rates
    ed_pqi_state['ed_visits_per_1k'] = (ed_pqi_state['ed_visits'] / ed_pqi_state['population']) * 1000
    ed_pqi_state['pqi_rate_per_100k'] = (ed_pqi_state['pqi_sum_count'] / ed_pqi_state['pqi_sum_population']) * 100000
    
    print(f"\n--- Statewide ED and PQI Trends ---")
    print(ed_pqi_state[['year', 'ed_visits_per_1k', 'pqi_rate_per_100k']].to_string(index=False))
    
    # Create combined time series figure
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: ED and PQI trends (dual axis)
    ax1 = axes[0, 0]
    ax1_twin = ax1.twinx()
    
    line1 = ax1.plot(ed_pqi_state['year'], ed_pqi_state['ed_visits_per_1k'], 'b-o', 
                     linewidth=2, markersize=6, label='ED Visits per 1k')
    line2 = ax1_twin.plot(ed_pqi_state['year'], ed_pqi_state['pqi_rate_per_100k'], 'r-s', 
                          linewidth=2, markersize=6, label='PQI Rate per 100k')
    
    ax1.set_xlabel('Year', fontsize=11)
    ax1.set_ylabel('ED Visits per 1,000', color='blue', fontsize=11)
    ax1_twin.set_ylabel('PQI Rate per 100,000', color='red', fontsize=11)
    ax1.set_title('ED Utilization and Preventable Hospitalizations\n(Statewide Trends)', fontsize=12, fontweight='bold')
    ax1.axvline(x=2020, color='gray', linestyle='--', alpha=0.5, label='COVID-19')
    ax1.grid(True, alpha=0.3)
    
    # Combine legends
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='upper left')
    
    # Plot 2: Year-over-year changes
    ax2 = axes[0, 1]
    ed_pqi_state['ed_change'] = ed_pqi_state['ed_visits_per_1k'].pct_change() * 100
    ed_pqi_state['pqi_change'] = ed_pqi_state['pqi_rate_per_100k'].pct_change() * 100
    
    ax2.bar(ed_pqi_state['year'][1:] - 0.2, ed_pqi_state['ed_change'][1:], width=0.4, 
            label='ED Change (%)', color='steelblue', edgecolor='black')
    ax2.bar(ed_pqi_state['year'][1:] + 0.2, ed_pqi_state['pqi_change'][1:], width=0.4,
            label='PQI Change (%)', color='firebrick', edgecolor='black')
    ax2.axhline(y=0, color='black', linewidth=0.5)
    ax2.set_xlabel('Year', fontsize=11)
    ax2.set_ylabel('Year-over-Year Change (%)', fontsize=11)
    ax2.set_title('Annual Changes in ED and PQI', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Plot 3: Correlation over time (scatter with years labeled)
    ax3 = axes[1, 0]
    scatter = ax3.scatter(ed_pqi_state['ed_visits_per_1k'], ed_pqi_state['pqi_rate_per_100k'],
                          c=ed_pqi_state['year'], cmap='viridis', s=100, edgecolors='black', linewidth=1)
    for idx, row in ed_pqi_state.iterrows():
        ax3.annotate(str(int(row['year'])), (row['ed_visits_per_1k'], row['pqi_rate_per_100k']),
                     fontsize=9, ha='left', va='bottom')
    ax3.set_xlabel('ED Visits per 1,000', fontsize=11)
    ax3.set_ylabel('PQI Rate per 100,000', fontsize=11)
    ax3.set_title('ED vs PQI Over Time (State-Level)', fontsize=12, fontweight='bold')
    cbar = plt.colorbar(scatter, ax=ax3)
    cbar.set_label('Year')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Indexed trends (2012 = 100)
    ax4 = axes[1, 1]
    base_ed = ed_pqi_state['ed_visits_per_1k'].iloc[0]
    base_pqi = ed_pqi_state['pqi_rate_per_100k'].iloc[0]
    
    ed_pqi_state['ed_indexed'] = (ed_pqi_state['ed_visits_per_1k'] / base_ed) * 100
    ed_pqi_state['pqi_indexed'] = (ed_pqi_state['pqi_rate_per_100k'] / base_pqi) * 100
    
    ax4.plot(ed_pqi_state['year'], ed_pqi_state['ed_indexed'], 'b-o', linewidth=2, markersize=6, label='ED Visits')
    ax4.plot(ed_pqi_state['year'], ed_pqi_state['pqi_indexed'], 'r-s', linewidth=2, markersize=6, label='PQI Rate')
    ax4.axhline(y=100, color='gray', linestyle='--', alpha=0.5)
    ax4.axvline(x=2020, color='gray', linestyle='--', alpha=0.5, label='COVID-19')
    ax4.set_xlabel('Year', fontsize=11)
    ax4.set_ylabel('Index (2012 = 100)', fontsize=11)
    ax4.set_title('Indexed Trends: ED and PQI\n(Base Year 2012 = 100)', fontsize=12, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('outputs/figures/ed_pqi_time_series.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Saved: outputs/figures/ed_pqi_time_series.png")
    
    # Calculate correlation
    corr = ed_pqi_state['ed_visits_per_1k'].corr(ed_pqi_state['pqi_rate_per_100k'])
    print(f"\nState-level correlation (ED vs PQI over time): {corr:.3f}")
    
    # Save panel data
    ed_pqi_panel.to_csv('outputs/data/ed_pqi_county_year_panel.csv', index=False)
    ed_pqi_state.to_csv('outputs/data/ed_pqi_statewide_trends.csv', index=False)
    print("✓ Saved: outputs/data/ed_pqi_county_year_panel.csv")
    print("✓ Saved: outputs/data/ed_pqi_statewide_trends.csv")
    
else:
    print("Cannot create extended time series - missing ED or PQI data.")

In [ ]:
# ED Analysis Summary
print("=" * 60)
print("ED ENCOUNTERS ANALYSIS SUMMARY")
print("=" * 60)

ed_summary = """
## ED Encounters Analysis Summary

### Data Overview
- **Source:** California OSHPD/HCAI Emergency Department Encounters
- **Years:** 2012-2023 (12 years of time series data)
- **Coverage:** ~55 counties with hospital ED services
- **Metrics:** ED visits, ED admissions, admission rates

### Key Findings

#### 1. Statewide Trends (2012-2023)
- ED utilization declined significantly during COVID-19 (2020)
- Post-pandemic recovery shows return to pre-COVID levels
- ED admission rates have remained relatively stable

#### 2. County Variation
- Large variation in ED utilization across California counties
- Rural counties tend to have higher ED visit rates per capita
- Urban counties have higher total volumes but lower per-capita rates

#### 3. Relationship with Medi-Cal Deserts
- Desert counties show HIGHER ED utilization (indicator of access barriers)
- Higher ED use correlates with higher PQI rates
- ED may serve as proxy for primary care access gaps

#### 4. Regression Results
- Medi-Cal share positively associated with ED utilization
- ED utilization partially mediates Medi-Cal → PQI relationship
- Controlling for ED reduces but does not eliminate Medi-Cal effect

### New Output Files

#### Data Files
- `ed_encounters_county_year.csv` - County-year ED data
- `ed_encounters_with_rates.csv` - With per-capita rates
- `ed_statewide_trends.csv` - State-level trends
- `ed_pqi_county_year_panel.csv` - ED + PQI merged panel
- `ed_pqi_statewide_trends.csv` - State-level ED+PQI trends
- `ca_master_panel_with_ed.csv` - Full panel with ED data

#### Figures
- `ed_trends_2012_2023.png` - Statewide ED time series
- `ed_county_comparison.png` - County variation
- `ed_desert_comparison.png` - Desert vs non-desert
- `ed_pqi_time_series.png` - ED and PQI trends together

#### Tables
- `ed_regression_results.csv` - ED regression results

### Implications
1. ED utilization is a useful proxy for primary care access
2. High ED rates in desert counties suggest unmet primary care needs
3. Time series available from 2012-2023 provides longer window than Medi-Cal enrollment data
4. ED data can help identify counties where primary care expansion is most needed
"""

print(ed_summary)

# Save summary
with open('outputs/ED_ANALYSIS_SUMMARY.md', 'w') as f:
    f.write(ed_summary)
print("\n✓ Saved: outputs/ED_ANALYSIS_SUMMARY.md")

print("\n" + "=" * 60)
print("ED ENCOUNTERS INTEGRATION COMPLETE")
print("=" * 60)

## 16) ED Intensity & System Strain Analysis

This section implements comprehensive ED intensity regressions to test whether:
1. **Mechanism Step 1:** High Medi-Cal share → Higher ED utilization (system strain)
2. **Mechanism Step 2:** ED utilization → Higher PQI (preventable hospitalizations)
3. **Interaction Effect:** Does ED strain amplify the Medi-Cal → PQI relationship?

All models run in **both unweighted and population-weighted** versions.

In [ ]:
# Modular Functions for ED Intensity Analysis
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS, WLS

def load_ed_data(filepath='encounters.csv'):
    """Load ED encounters data."""
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        print(f"✓ Loaded {filepath}: {df.shape}")
        return df
    else:
        print(f"ERROR: {filepath} not found!")
        return None

def map_facility_to_county(df):
    """Map facility to county using county_name field."""
    df = df.copy()
    df['county_clean'] = df['county_name'].apply(standardize_county_name)
    return df

def aggregate_ed_to_county_year(df, crosswalk):
    """Aggregate ED encounters to county-year level."""
    # Pivot to get ED_Visit and ED_Admit as columns
    pivot = df.pivot_table(
        index=['year', 'county_clean'],
        columns='type',
        values='count',
        aggfunc='sum'
    ).reset_index()
    
    pivot.columns.name = None
    pivot = pivot.rename(columns={'ED_Admit': 'ed_admissions', 'ED_Visit': 'ed_visits'})
    pivot['ed_admissions'] = pivot['ed_admissions'].fillna(0)
    pivot['ed_visits'] = pivot['ed_visits'].fillna(0)
    pivot['ed_total'] = pivot['ed_visits'] + pivot['ed_admissions']
    pivot['ed_admit_share'] = pivot['ed_admissions'] / pivot['ed_total']
    pivot['ed_admit_share'] = pivot['ed_admit_share'].replace([np.inf, -np.inf], np.nan)
    
    # Merge with crosswalk
    result = pivot.merge(crosswalk[['county_name_clean', 'fips5']], 
                         left_on='county_clean', right_on='county_name_clean', how='left')
    result = result[result['fips5'].notna()].copy()
    result = result[['fips5', 'year', 'ed_visits', 'ed_admissions', 'ed_total', 'ed_admit_share']]
    
    return result

def merge_ed_to_master(master_panel, ed_county_year, pop_year):
    """Merge ED data with master panel and compute per-capita rates."""
    # Merge ED with population
    ed_with_pop = ed_county_year.merge(pop_year[['fips5', 'year', 'population']], 
                                        on=['fips5', 'year'], how='left')
    
    # Calculate per-capita rates
    ed_with_pop['ed_visits_per_1k'] = (ed_with_pop['ed_visits'] / ed_with_pop['population']) * 1000
    ed_with_pop['ed_admits_per_1k'] = (ed_with_pop['ed_admissions'] / ed_with_pop['population']) * 1000
    ed_with_pop['log_ed_visits_per_1k'] = np.log(ed_with_pop['ed_visits_per_1k'] + 0.1)
    
    # Merge with master panel
    ed_vars = ['fips5', 'year', 'ed_visits', 'ed_admissions', 'ed_total', 
               'ed_visits_per_1k', 'ed_admits_per_1k', 'ed_admit_share', 'log_ed_visits_per_1k']
    
    merged = master_panel.merge(ed_with_pop[ed_vars], on=['fips5', 'year'], how='left')
    
    return merged, ed_with_pop

def run_regression(Y, X, weights=None, cluster_var=None):
    """Run OLS or WLS regression with robust SEs."""
    X_const = sm.add_constant(X)
    
    if weights is not None:
        model = WLS(Y, X_const, weights=weights)
    else:
        model = OLS(Y, X_const)
    
    results = model.fit(cov_type='HC1')
    return results

def format_results(results, model_name, dep_var, weighted=False):
    """Format regression results for output."""
    return {
        'Model': model_name,
        'DV': dep_var,
        'Weighted': 'Yes' if weighted else 'No',
        'N': int(results.nobs),
        'R2': results.rsquared,
        'Adj_R2': results.rsquared_adj
    }

print("✓ Modular functions defined for ED intensity analysis")

In [ ]:
# Prepare Data for ED Intensity Regressions
print("=" * 70)
print("PREPARING DATA FOR ED INTENSITY REGRESSIONS")
print("=" * 70)

# Load and process ED data using modular functions
df_ed_raw = load_ed_data('encounters.csv')

if df_ed_raw is not None:
    # Map facilities to counties
    df_ed_mapped = map_facility_to_county(df_ed_raw)
    
    # Aggregate to county-year
    ed_county_year = aggregate_ed_to_county_year(df_ed_mapped, crosswalk_clean)
    print(f"\n✓ ED county-year dataset: {ed_county_year.shape}")
    print(f"  Year range: {ed_county_year['year'].min()} - {ed_county_year['year'].max()}")
    print(f"  Unique counties: {ed_county_year['fips5'].nunique()}")
    
    # Save ED county-year dataset
    ed_county_year.to_csv('outputs/data/ed_county_year.csv', index=False)
    print(f"\n✓ Saved: outputs/data/ed_county_year.csv")
    
    # Merge with master panel
    master_with_ed_full, ed_with_pop = merge_ed_to_master(master_panel, ed_county_year, pop_year)
    print(f"\n✓ Master panel with ED: {master_with_ed_full.shape}")
    
    # Save full panel
    master_with_ed_full.to_csv('outputs/data/ca_master_panel_with_ed.csv', index=False)
    print(f"✓ Saved: outputs/data/ca_master_panel_with_ed.csv")
    
    # Create 2020 cross-section
    cross_section_2020 = master_with_ed_full[master_with_ed_full['year'] == 2020].copy()
    
    # Add Medi-Cal threshold indicator
    cross_section_2020['medi_cal_ge_30'] = (cross_section_2020['medi_cal_share'] >= 0.30).astype(int)
    
    print(f"\n✓ 2020 cross-section: {cross_section_2020.shape}")
    print(f"  Counties: {cross_section_2020['fips5'].nunique()}")
    print(f"  Counties with ≥30% Medi-Cal: {cross_section_2020['medi_cal_ge_30'].sum()}")
    
    # Save cross-section
    cross_section_2020.to_csv('outputs/data/ca_cross_section_2020_with_ed.csv', index=False)
    print(f"✓ Saved: outputs/data/ca_cross_section_2020_with_ed.csv")
    
    # Summary statistics for ED variables
    print(f"\n--- ED Variables Summary (2020) ---")
    ed_vars_summary = ['ed_visits_per_1k', 'ed_admits_per_1k', 'ed_admit_share', 'log_ed_visits_per_1k']
    print(cross_section_2020[ed_vars_summary].describe().round(2))
else:
    cross_section_2020 = None
    print("ERROR: Cannot prepare data - ED file not found")

In [ ]:
# MECHANISM STEP 1: ED Intensity as Outcome
# Does Medi-Cal share predict higher ED utilization?
print("=" * 70)
print("MECHANISM STEP 1: ED INTENSITY AS OUTCOME")
print("=" * 70)

if cross_section_2020 is not None:
    # Define controls
    control_vars = ['poverty_pct', 'unemp_pct', 'bachelors_pct', 'age65_pct', 'hispanic_pct']
    available_controls = [v for v in control_vars if v in cross_section_2020.columns]
    print(f"Controls: {available_controls}")
    
    # Prepare analysis data - drop missing
    analysis_vars = ['ed_visits_per_1k', 'log_ed_visits_per_1k', 'ed_admit_share',
                     'medi_cal_share', 'medi_cal_ge_30', 'population'] + available_controls
    analysis_df = cross_section_2020.dropna(subset=['ed_visits_per_1k', 'medi_cal_share'] + available_controls).copy()
    print(f"\nAnalysis sample: {len(analysis_df)} counties")
    
    # Store all results
    ed_results_list = []
    
    # ===== MODEL ED1a: ED visits ~ Medi-Cal share (continuous) - UNWEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL ED1a: ED Visits per 1k ~ Medi-Cal Share (Continuous)")
    print("UNWEIGHTED")
    print("=" * 60)
    
    Y_ed1a = analysis_df['ed_visits_per_1k']
    X_ed1a = analysis_df[['medi_cal_share'] + available_controls]
    
    model_ed1a = run_regression(Y_ed1a, X_ed1a)
    print(model_ed1a.summary2().tables[1])
    
    ed_results_list.append({
        'Model': 'ED1a', 'DV': 'ed_visits_per_1k', 'Key_Predictor': 'medi_cal_share (continuous)',
        'Weighted': 'No', 'Key_Coef': model_ed1a.params.get('medi_cal_share', np.nan),
        'Key_SE': model_ed1a.bse.get('medi_cal_share', np.nan),
        'Key_Pval': model_ed1a.pvalues.get('medi_cal_share', np.nan),
        'N': int(model_ed1a.nobs), 'R2': model_ed1a.rsquared
    })
    
    # ===== MODEL ED1a_W: ED visits ~ Medi-Cal share (continuous) - WEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL ED1a_W: ED Visits per 1k ~ Medi-Cal Share (Continuous)")
    print("POPULATION-WEIGHTED")
    print("=" * 60)
    
    weights = analysis_df['population']
    model_ed1a_w = run_regression(Y_ed1a, X_ed1a, weights=weights)
    print(model_ed1a_w.summary2().tables[1])
    
    ed_results_list.append({
        'Model': 'ED1a_W', 'DV': 'ed_visits_per_1k', 'Key_Predictor': 'medi_cal_share (continuous)',
        'Weighted': 'Yes', 'Key_Coef': model_ed1a_w.params.get('medi_cal_share', np.nan),
        'Key_SE': model_ed1a_w.bse.get('medi_cal_share', np.nan),
        'Key_Pval': model_ed1a_w.pvalues.get('medi_cal_share', np.nan),
        'N': int(model_ed1a_w.nobs), 'R2': model_ed1a_w.rsquared
    })
    
    # ===== MODEL ED1b: ED visits ~ Medi-Cal ≥30% threshold - UNWEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL ED1b: ED Visits per 1k ~ I(Medi-Cal ≥ 30%)")
    print("UNWEIGHTED")
    print("=" * 60)
    
    X_ed1b = analysis_df[['medi_cal_ge_30'] + available_controls]
    model_ed1b = run_regression(Y_ed1a, X_ed1b)
    print(model_ed1b.summary2().tables[1])
    
    ed_results_list.append({
        'Model': 'ED1b', 'DV': 'ed_visits_per_1k', 'Key_Predictor': 'medi_cal_ge_30 (≥30%)',
        'Weighted': 'No', 'Key_Coef': model_ed1b.params.get('medi_cal_ge_30', np.nan),
        'Key_SE': model_ed1b.bse.get('medi_cal_ge_30', np.nan),
        'Key_Pval': model_ed1b.pvalues.get('medi_cal_ge_30', np.nan),
        'N': int(model_ed1b.nobs), 'R2': model_ed1b.rsquared
    })
    
    # ===== MODEL ED1b_W: ED visits ~ Medi-Cal ≥30% threshold - WEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL ED1b_W: ED Visits per 1k ~ I(Medi-Cal ≥ 30%)")
    print("POPULATION-WEIGHTED")
    print("=" * 60)
    
    model_ed1b_w = run_regression(Y_ed1a, X_ed1b, weights=weights)
    print(model_ed1b_w.summary2().tables[1])
    
    ed_results_list.append({
        'Model': 'ED1b_W', 'DV': 'ed_visits_per_1k', 'Key_Predictor': 'medi_cal_ge_30 (≥30%)',
        'Weighted': 'Yes', 'Key_Coef': model_ed1b_w.params.get('medi_cal_ge_30', np.nan),
        'Key_SE': model_ed1b_w.bse.get('medi_cal_ge_30', np.nan),
        'Key_Pval': model_ed1b_w.pvalues.get('medi_cal_ge_30', np.nan),
        'N': int(model_ed1b_w.nobs), 'R2': model_ed1b_w.rsquared
    })
    
    # ===== MODEL ED2: Log ED visits - UNWEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL ED2: Log(ED Visits per 1k) ~ Medi-Cal Share")
    print("UNWEIGHTED")
    print("=" * 60)
    
    Y_ed2 = analysis_df['log_ed_visits_per_1k']
    X_ed2 = analysis_df[['medi_cal_share'] + available_controls]
    model_ed2 = run_regression(Y_ed2, X_ed2)
    print(model_ed2.summary2().tables[1])
    
    ed_results_list.append({
        'Model': 'ED2', 'DV': 'log(ed_visits_per_1k)', 'Key_Predictor': 'medi_cal_share (continuous)',
        'Weighted': 'No', 'Key_Coef': model_ed2.params.get('medi_cal_share', np.nan),
        'Key_SE': model_ed2.bse.get('medi_cal_share', np.nan),
        'Key_Pval': model_ed2.pvalues.get('medi_cal_share', np.nan),
        'N': int(model_ed2.nobs), 'R2': model_ed2.rsquared
    })
    
    # ===== MODEL ED2_W: Log ED visits - WEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL ED2_W: Log(ED Visits per 1k) ~ Medi-Cal Share")
    print("POPULATION-WEIGHTED")
    print("=" * 60)
    
    model_ed2_w = run_regression(Y_ed2, X_ed2, weights=weights)
    print(model_ed2_w.summary2().tables[1])
    
    ed_results_list.append({
        'Model': 'ED2_W', 'DV': 'log(ed_visits_per_1k)', 'Key_Predictor': 'medi_cal_share (continuous)',
        'Weighted': 'Yes', 'Key_Coef': model_ed2_w.params.get('medi_cal_share', np.nan),
        'Key_SE': model_ed2_w.bse.get('medi_cal_share', np.nan),
        'Key_Pval': model_ed2_w.pvalues.get('medi_cal_share', np.nan),
        'N': int(model_ed2_w.nobs), 'R2': model_ed2_w.rsquared
    })
    
    # ===== OPTIONAL: ED Admission Share =====
    print("\n" + "=" * 60)
    print("MODEL ED3: ED Admission Share ~ Medi-Cal Share (Optional)")
    print("UNWEIGHTED")
    print("=" * 60)
    
    Y_ed3 = analysis_df['ed_admit_share'].dropna()
    X_ed3 = analysis_df.loc[Y_ed3.index, ['medi_cal_share'] + available_controls]
    model_ed3 = run_regression(Y_ed3, X_ed3)
    print(model_ed3.summary2().tables[1])
    
    ed_results_list.append({
        'Model': 'ED3', 'DV': 'ed_admit_share', 'Key_Predictor': 'medi_cal_share (continuous)',
        'Weighted': 'No', 'Key_Coef': model_ed3.params.get('medi_cal_share', np.nan),
        'Key_SE': model_ed3.bse.get('medi_cal_share', np.nan),
        'Key_Pval': model_ed3.pvalues.get('medi_cal_share', np.nan),
        'N': int(model_ed3.nobs), 'R2': model_ed3.rsquared
    })
    
    # Save ED intensity regression results
    ed_results_df = pd.DataFrame(ed_results_list)
    ed_results_df.to_csv('outputs/tables/reg_ed_intensity_2020.csv', index=False)
    print(f"\n✓ Saved: outputs/tables/reg_ed_intensity_2020.csv")
    
    print("\n" + "=" * 60)
    print("ED INTENSITY REGRESSION SUMMARY")
    print("=" * 60)
    print(ed_results_df.to_string(index=False))
else:
    ed_results_df = None
    print("Cannot run ED intensity regressions - data not available")

In [ ]:
# MECHANISM STEP 2: PQI as Outcome with ED Added
# Does ED utilization mediate or amplify the Medi-Cal → PQI relationship?
print("=" * 70)
print("MECHANISM STEP 2: PQI AS OUTCOME WITH ED INTENSITY")
print("=" * 70)

if cross_section_2020 is not None and 'pqi_mean_rate' in analysis_df.columns:
    # Prepare PQI analysis data
    pqi_vars = ['pqi_mean_rate', 'ed_visits_per_1k', 'medi_cal_share', 'medi_cal_ge_30', 'population']
    pqi_analysis_df = analysis_df.dropna(subset=['pqi_mean_rate', 'ed_visits_per_1k', 'medi_cal_share'] + available_controls).copy()
    
    # Create interaction term
    pqi_analysis_df['mc_x_ed'] = pqi_analysis_df['medi_cal_share'] * pqi_analysis_df['ed_visits_per_1k']
    
    print(f"PQI Analysis sample: {len(pqi_analysis_df)} counties")
    
    # Store PQI results
    pqi_results_list = []
    
    # ===== MODEL PQI1: Baseline (no ED) - UNWEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL PQI1: PQI ~ Medi-Cal Share + Controls (Baseline)")
    print("UNWEIGHTED")
    print("=" * 60)
    
    Y_pqi = pqi_analysis_df['pqi_mean_rate']
    X_pqi1 = pqi_analysis_df[['medi_cal_share'] + available_controls]
    weights_pqi = pqi_analysis_df['population']
    
    model_pqi1 = run_regression(Y_pqi, X_pqi1)
    print(model_pqi1.summary2().tables[1])
    
    mc_coef_baseline = model_pqi1.params.get('medi_cal_share', np.nan)
    
    pqi_results_list.append({
        'Model': 'PQI1', 'DV': 'pqi_mean_rate', 'Specification': 'Baseline (no ED)',
        'Weighted': 'No', 'MC_Coef': mc_coef_baseline,
        'MC_SE': model_pqi1.bse.get('medi_cal_share', np.nan),
        'MC_Pval': model_pqi1.pvalues.get('medi_cal_share', np.nan),
        'ED_Coef': np.nan, 'ED_Pval': np.nan,
        'N': int(model_pqi1.nobs), 'R2': model_pqi1.rsquared
    })
    
    # ===== MODEL PQI1_W: Baseline - WEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL PQI1_W: PQI ~ Medi-Cal Share + Controls (Baseline)")
    print("POPULATION-WEIGHTED")
    print("=" * 60)
    
    model_pqi1_w = run_regression(Y_pqi, X_pqi1, weights=weights_pqi)
    print(model_pqi1_w.summary2().tables[1])
    
    pqi_results_list.append({
        'Model': 'PQI1_W', 'DV': 'pqi_mean_rate', 'Specification': 'Baseline (no ED)',
        'Weighted': 'Yes', 'MC_Coef': model_pqi1_w.params.get('medi_cal_share', np.nan),
        'MC_SE': model_pqi1_w.bse.get('medi_cal_share', np.nan),
        'MC_Pval': model_pqi1_w.pvalues.get('medi_cal_share', np.nan),
        'ED_Coef': np.nan, 'ED_Pval': np.nan,
        'N': int(model_pqi1_w.nobs), 'R2': model_pqi1_w.rsquared
    })
    
    # ===== MODEL PQI2: With ED - UNWEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL PQI2: PQI ~ Medi-Cal Share + ED Visits + Controls")
    print("UNWEIGHTED")
    print("=" * 60)
    
    X_pqi2 = pqi_analysis_df[['medi_cal_share', 'ed_visits_per_1k'] + available_controls]
    model_pqi2 = run_regression(Y_pqi, X_pqi2)
    print(model_pqi2.summary2().tables[1])
    
    mc_coef_with_ed = model_pqi2.params.get('medi_cal_share', np.nan)
    pct_change = ((mc_coef_with_ed - mc_coef_baseline) / mc_coef_baseline * 100) if mc_coef_baseline != 0 else np.nan
    
    print(f"\n** Coefficient Change when ED added: {pct_change:.1f}% **")
    
    pqi_results_list.append({
        'Model': 'PQI2', 'DV': 'pqi_mean_rate', 'Specification': 'With ED',
        'Weighted': 'No', 'MC_Coef': mc_coef_with_ed,
        'MC_SE': model_pqi2.bse.get('medi_cal_share', np.nan),
        'MC_Pval': model_pqi2.pvalues.get('medi_cal_share', np.nan),
        'ED_Coef': model_pqi2.params.get('ed_visits_per_1k', np.nan),
        'ED_Pval': model_pqi2.pvalues.get('ed_visits_per_1k', np.nan),
        'N': int(model_pqi2.nobs), 'R2': model_pqi2.rsquared
    })
    
    # ===== MODEL PQI2_W: With ED - WEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL PQI2_W: PQI ~ Medi-Cal Share + ED Visits + Controls")
    print("POPULATION-WEIGHTED")
    print("=" * 60)
    
    model_pqi2_w = run_regression(Y_pqi, X_pqi2, weights=weights_pqi)
    print(model_pqi2_w.summary2().tables[1])
    
    pqi_results_list.append({
        'Model': 'PQI2_W', 'DV': 'pqi_mean_rate', 'Specification': 'With ED',
        'Weighted': 'Yes', 'MC_Coef': model_pqi2_w.params.get('medi_cal_share', np.nan),
        'MC_SE': model_pqi2_w.bse.get('medi_cal_share', np.nan),
        'MC_Pval': model_pqi2_w.pvalues.get('medi_cal_share', np.nan),
        'ED_Coef': model_pqi2_w.params.get('ed_visits_per_1k', np.nan),
        'ED_Pval': model_pqi2_w.pvalues.get('ed_visits_per_1k', np.nan),
        'N': int(model_pqi2_w.nobs), 'R2': model_pqi2_w.rsquared
    })
    
    # ===== MODEL PQI3: With ED Interaction - UNWEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL PQI3: PQI ~ MC + ED + (MC × ED) + Controls (Interaction)")
    print("UNWEIGHTED")
    print("=" * 60)
    
    X_pqi3 = pqi_analysis_df[['medi_cal_share', 'ed_visits_per_1k', 'mc_x_ed'] + available_controls]
    model_pqi3 = run_regression(Y_pqi, X_pqi3)
    print(model_pqi3.summary2().tables[1])
    
    interaction_coef = model_pqi3.params.get('mc_x_ed', np.nan)
    interaction_pval = model_pqi3.pvalues.get('mc_x_ed', np.nan)
    
    print(f"\n** Interaction (MC × ED): {interaction_coef:.4f} (p={interaction_pval:.4f}) **")
    if interaction_pval < 0.05:
        print("   → Significant interaction: ED strain amplifies Medi-Cal effect")
    else:
        print("   → No significant interaction")
    
    pqi_results_list.append({
        'Model': 'PQI3', 'DV': 'pqi_mean_rate', 'Specification': 'With Interaction',
        'Weighted': 'No', 'MC_Coef': model_pqi3.params.get('medi_cal_share', np.nan),
        'MC_SE': model_pqi3.bse.get('medi_cal_share', np.nan),
        'MC_Pval': model_pqi3.pvalues.get('medi_cal_share', np.nan),
        'ED_Coef': model_pqi3.params.get('ed_visits_per_1k', np.nan),
        'ED_Pval': model_pqi3.pvalues.get('ed_visits_per_1k', np.nan),
        'Interaction_Coef': interaction_coef, 'Interaction_Pval': interaction_pval,
        'N': int(model_pqi3.nobs), 'R2': model_pqi3.rsquared
    })
    
    # ===== MODEL PQI3_W: With ED Interaction - WEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL PQI3_W: PQI ~ MC + ED + (MC × ED) + Controls (Interaction)")
    print("POPULATION-WEIGHTED")
    print("=" * 60)
    
    model_pqi3_w = run_regression(Y_pqi, X_pqi3, weights=weights_pqi)
    print(model_pqi3_w.summary2().tables[1])
    
    pqi_results_list.append({
        'Model': 'PQI3_W', 'DV': 'pqi_mean_rate', 'Specification': 'With Interaction',
        'Weighted': 'Yes', 'MC_Coef': model_pqi3_w.params.get('medi_cal_share', np.nan),
        'MC_SE': model_pqi3_w.bse.get('medi_cal_share', np.nan),
        'MC_Pval': model_pqi3_w.pvalues.get('medi_cal_share', np.nan),
        'ED_Coef': model_pqi3_w.params.get('ed_visits_per_1k', np.nan),
        'ED_Pval': model_pqi3_w.pvalues.get('ed_visits_per_1k', np.nan),
        'Interaction_Coef': model_pqi3_w.params.get('mc_x_ed', np.nan),
        'Interaction_Pval': model_pqi3_w.pvalues.get('mc_x_ed', np.nan),
        'N': int(model_pqi3_w.nobs), 'R2': model_pqi3_w.rsquared
    })
    
    # ===== MODEL PQI_THR: With ≥30% threshold - UNWEIGHTED =====
    print("\n" + "=" * 60)
    print("MODEL PQI_THR: PQI ~ I(MC ≥ 30%) + ED + Controls")
    print("UNWEIGHTED")
    print("=" * 60)
    
    X_pqi_thr = pqi_analysis_df[['medi_cal_ge_30', 'ed_visits_per_1k'] + available_controls]
    model_pqi_thr = run_regression(Y_pqi, X_pqi_thr)
    print(model_pqi_thr.summary2().tables[1])
    
    pqi_results_list.append({
        'Model': 'PQI_THR', 'DV': 'pqi_mean_rate', 'Specification': '≥30% Threshold + ED',
        'Weighted': 'No', 'MC_Coef': model_pqi_thr.params.get('medi_cal_ge_30', np.nan),
        'MC_SE': model_pqi_thr.bse.get('medi_cal_ge_30', np.nan),
        'MC_Pval': model_pqi_thr.pvalues.get('medi_cal_ge_30', np.nan),
        'ED_Coef': model_pqi_thr.params.get('ed_visits_per_1k', np.nan),
        'ED_Pval': model_pqi_thr.pvalues.get('ed_visits_per_1k', np.nan),
        'N': int(model_pqi_thr.nobs), 'R2': model_pqi_thr.rsquared
    })
    
    # Save PQI with ED regression results
    pqi_results_df = pd.DataFrame(pqi_results_list)
    pqi_results_df.to_csv('outputs/tables/reg_pqi_with_ed_2020.csv', index=False)
    print(f"\n✓ Saved: outputs/tables/reg_pqi_with_ed_2020.csv")
    
    print("\n" + "=" * 60)
    print("PQI WITH ED REGRESSION SUMMARY")
    print("=" * 60)
    print(pqi_results_df[['Model', 'Specification', 'Weighted', 'MC_Coef', 'MC_Pval', 'ED_Coef', 'ED_Pval', 'R2']].to_string(index=False))
else:
    pqi_results_df = None
    print("Cannot run PQI regressions - data not available")

In [ ]:
# Diagnostic Plots for ED Intensity Analysis
print("=" * 70)
print("DIAGNOSTIC PLOTS: ED INTENSITY ANALYSIS")
print("=" * 70)

if cross_section_2020 is not None:
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # ===== Plot 1: Histogram of ED visits per 1k =====
    ax1 = axes[0, 0]
    ed_data = analysis_df['ed_visits_per_1k'].dropna()
    ax1.hist(ed_data, bins=25, edgecolor='black', alpha=0.7, color='steelblue')
    ax1.axvline(ed_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {ed_data.mean():.1f}')
    ax1.axvline(ed_data.median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {ed_data.median():.1f}')
    ax1.set_xlabel('ED Visits per 1,000 Population', fontsize=11)
    ax1.set_ylabel('Frequency', fontsize=11)
    ax1.set_title('Distribution of ED Utilization (2020)', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # ===== Plot 2: Medi-Cal share vs ED visits per 1k =====
    ax2 = axes[0, 1]
    x_mc = analysis_df['medi_cal_share']
    y_ed = analysis_df['ed_visits_per_1k']
    
    ax2.scatter(x_mc, y_ed, alpha=0.6, s=60, c='steelblue', edgecolors='black', linewidth=0.5)
    
    # Fit line
    mask = x_mc.notna() & y_ed.notna()
    z = np.polyfit(x_mc[mask], y_ed[mask], 1)
    p = np.poly1d(z)
    x_line = np.linspace(x_mc.min(), x_mc.max(), 100)
    ax2.plot(x_line, p(x_line), 'r--', linewidth=2, label=f'Fitted: y={z[0]:.1f}x+{z[1]:.1f}')
    
    # Label top outliers
    top_outliers = analysis_df.nlargest(5, 'ed_visits_per_1k')
    for idx, row in top_outliers.iterrows():
        county = crosswalk_clean[crosswalk_clean['fips5'] == row['fips5']]['county_name_clean'].values
        if len(county) > 0:
            ax2.annotate(county[0], (row['medi_cal_share'], row['ed_visits_per_1k']), 
                        fontsize=8, alpha=0.8, ha='left')
    
    ax2.axvline(x=0.30, color='green', linestyle=':', linewidth=2, alpha=0.7, label='30% Threshold')
    ax2.set_xlabel('Medi-Cal Share', fontsize=11)
    ax2.set_ylabel('ED Visits per 1,000 Population', fontsize=11)
    ax2.set_title('Medi-Cal Share vs ED Utilization', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # ===== Plot 3: ED visits vs PQI =====
    ax3 = axes[1, 0]
    x_ed = analysis_df['ed_visits_per_1k']
    y_pqi = analysis_df['pqi_mean_rate']
    
    ax3.scatter(x_ed, y_pqi, alpha=0.6, s=60, c='firebrick', edgecolors='black', linewidth=0.5)
    
    # Fit line
    mask = x_ed.notna() & y_pqi.notna()
    z = np.polyfit(x_ed[mask], y_pqi[mask], 1)
    p = np.poly1d(z)
    x_line = np.linspace(x_ed.min(), x_ed.max(), 100)
    ax3.plot(x_line, p(x_line), 'b--', linewidth=2, label=f'Fitted: y={z[0]:.2f}x+{z[1]:.1f}')
    
    ax3.set_xlabel('ED Visits per 1,000 Population', fontsize=11)
    ax3.set_ylabel('PQI Mean Rate', fontsize=11)
    ax3.set_title('ED Utilization vs Preventable Hospitalizations', fontsize=12, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # ===== Plot 4: ED by Desert status =====
    ax4 = axes[1, 1]
    if 'desert_thr_def2' in analysis_df.columns:
        desert_col = 'desert_thr_def2'
    elif 'desert_q_def2' in analysis_df.columns:
        desert_col = 'desert_q_def2'
    else:
        desert_col = None
    
    if desert_col:
        non_desert = analysis_df[analysis_df[desert_col] == 0]['ed_visits_per_1k']
        desert = analysis_df[analysis_df[desert_col] == 1]['ed_visits_per_1k']
        
        positions = [1, 2]
        bp = ax4.boxplot([non_desert.dropna(), desert.dropna()], positions=positions, widths=0.6, patch_artist=True)
        bp['boxes'][0].set_facecolor('#2ca02c')
        bp['boxes'][1].set_facecolor('#d62728')
        
        ax4.set_xticks(positions)
        ax4.set_xticklabels(['Non-Desert', 'Desert'])
        ax4.set_ylabel('ED Visits per 1,000 Population', fontsize=11)
        ax4.set_title('ED Utilization: Desert vs Non-Desert Counties', fontsize=12, fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        
        # Add mean labels
        ax4.annotate(f'Mean: {non_desert.mean():.1f}', (1, non_desert.max()+10), ha='center', fontsize=9)
        ax4.annotate(f'Mean: {desert.mean():.1f}', (2, desert.max()+10), ha='center', fontsize=9)
    else:
        ax4.text(0.5, 0.5, 'Desert variable not available', ha='center', va='center', transform=ax4.transAxes)
    
    plt.tight_layout()
    plt.savefig('outputs/figures/ed_intensity_diagnostics.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Saved: outputs/figures/ed_intensity_diagnostics.png")

else:
    print("Cannot create diagnostic plots - data not available")